# Data Preparation for POMMES
This notebook contains the major routines used to prepare and provide input data for the fundamental power market model _POMMES_ <br>(<b>Po</b>wer <b>M</b>arket <b>M</b>odel <b>E</b>nergy and re<b>S</b>ources).

It provides input data for (oemof.solph components used for modeling given in brackets)
* conventional power plants (Transformers)
* interconnectors (Transformers; formerly Links)
* storages (GenericStorages)
* commodity sources (Sources)
* renewable generators (Sources / Transformers)
* power demand (Sinks)

Two separate data sets are created:
* one containing the power plant status as of 2017 with an update for 2019 and
* one containing an estimated exogeneously determined power plant status for 2030.

> **Outline:** The correspondend elements, named components, used in the framework [oemof.solph](https://github.com/oemof/oemof-solph) determine the outline of the given notebook.
> _NOTE: It is recommended to run the notebook sequentially since variables declared earlier may be needed later on, even in later sections._

> **Raw data sources:** The raw data is taken from different sources, mostly [OPSD](https://open-power-system-data.org/) and [ENTSO-E](https://transparency.entsoe.eu/). The major sources and assumptions are documented for the respective power system elements (components) in the following. For power plant projections, data from the lastest approved network development plan for electricity for Germany [NEP Strom 2030 as of 2019](https://www.netzentwicklungsplan.de/de/netzentwicklungsplaene/netzentwicklungsplan-2030-2019) as well as data from the [TYNDP 2018](https://tyndp.entsoe.eu/maps-data) of ENTSO-E for are used.<br>
An overview on the data licensing information can be found in the repository.

> **Tasks:** Besides extraction, cleaning and combination of data from the sources and assumptions, significant parts of the notebooks are routines for transferring the data to a data format that can be handled by the framework [oemof.solph](https://github.com/oemof/oemof-solph) which serves as a basis for the model _POMMES_.

> **Authors:** Corresponding authors: Yannick Werner, Johannes Kochems<br>Contributors: Leticia Encinas Rosa, Carla Spiller, Sophie Westphal, Julian Endres, Julien Faist, Timona Ghosh, Christian Fraatz, Robin Claus, Daniel Peschel, Conrad Nicklisch

# Package imports and settings

## Imports

User defined modules used:
* `tools`: a set of functions holding assisting routines; comprises functions for calculating (shortest) distances based on given GPS coordinates, for loading bidding zone shapes as well as ENTSO-E data, for assigning efficiencies as well as gradients and minimum loads and for setting NTC values
* `eeg_transformers`: functions for modeling RES clusters bidding at the negative market premium (routines for reading in data, data aggregation, clustering and assigning market values)
* `transformers_aggregation`: functions for clustering conventional power plants
* `hydro`: functions to load, preprocess and upsample hydro generation and filling rate data

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import statsmodels.api as sm

from re import search, IGNORECASE
from os import listdir
from os.path import isfile, join

import data_prep.tools as tools
import data_prep.eeg_transformers as eeg
import data_prep.transformer_aggregation as tf_agg
import data_prep.hydro as hydro

In [ ]:
# Path folders
path_folder_inputs = './raw_data_input/'
path_folder_outputs = './prepared_data/'

path_folder_interconnectors = 'interconnectors/'

path_folder_pp_public = 'powerplants_public/'
path_folder_pp_er = 'powerplants_er/'

path_folder_timeseries = 'timeseries/'
path_folder_hydro = 'hydro/'
path_folder_renewables = 'renewables/'

path_folder_marketzones = 'market_zones_scandinavia/'
path_folder_costs = 'costs/'
path_folder_assumptions = 'assumptions/'

# Filenames of input files
filename_opsd_de = 'KWlisteDE_OPSD_20191002.xlsx'
filename_opsd_de_new = 'conventional_power_plants_DE.csv'
filename_ppmatching_eu = 'powerplants_conventional_eu.xlsx'
filename_ppmatching_eu_new = 'powerplants_conventional_eu_20210105.csv'
filename_tyndp_eu = 'ENTSO Scenario 2018 Generation Capacities.xlsm'
filename_nep_de = 'KWliste_NEP2030_20191022.xlsx'

filename_techs_de = 'Technologies_ManRes_20170212.xlsx'
filename_peschel = 'KWliste_DP_20160304.xlsx'
filename_eff_de = 'Efficiencies_ManRes_20180918.xlsx'
filename_comm = 'transformers_commissioning.xlsx'
filename_decomm = 'transformers_decommissioning.xlsx'
filename_comm_decomm_BNetzA = 'BNetzA_Veroeff_ZuUndRueckbau_20200404.xlsx'
filename_KWSAL = 'KWSAL_20200415.csv'
filename_decomm_coal = 'transformers_decommissioning_coal.xlsx'
filename_new_built = 'transformers_new-built.csv'

filename_interconnectors_eu = 'Interconnectors_20191117.xlsx'
filename_interconnectors_timeseries = 'Interconnectors_timeseries_20191028.xlsx'

filename_renewables_nonfluctuating = 'capacity_factors_nonfluc_16082020.csv'
filename_eeg_powerplants = 'ee_powerplants_16082020.csv'
filename_market_values = 'netztransparenz_market_values_2017.csv'
filename_REcap_eu = 'REcapacitiesEU_IRENA_20201120.xlsx'
filename_REcap_DK = 'DK_capacities_20191120.xlsx'
filename_RES_DE_Prognos = 'Prognos_et_al_2020_RES_data.xlsx'
filename_RES_DE_EEG_2021 = 'EEG_2021_RES_capacity_targets.xlsx'
filename_onshore_tenders = 'Statistik_Onshore_20201117.xlsx' 
filename_solar_tenders = 'Statistik_Solar_20201117.xlsx'
filename_common_tenders = 'Statistik_GemAV_20201117.xlsx'
filename_PV_2018_01 = 'PV_DegressionsVergSaetze_Mai-Juli18.xlsx'
filename_PV_2019_01 = 'PV_DegressionsVergSaetze_11-01_19.xlsx'
filename_PV_2019_02 = 'PV_DegressionsVergSaetze_05-07_19.xlsx'
filename_PV_2020_01 = 'PV_DegressionsVergSaetze_11-01_20.xlsx'
filename_PV_2020_02 = 'PV_DegressionsVergSaetze_05-07_20.xlsx'
filename_PV_2021_01 = 'PV_DegressionsVergSaetze_11-01_21.xlsx'
filename_RES_cost_projections = 'RES_cost_projections.xlsx'
filename_RES_cost_ISE = "RES_cost_projections_ISE.csv"

filename_costs = 'cost_input_data_raw.xlsx'
filename_opsd_timeseries = 'Timeseries_OPSD_20191028.csv'
filename_entsoe_generation_de = 'entsoe_generation_DE_22072020.csv'
filename_assumptions = 'Assumptions.xlsx'
filename_emf = 'Emissionfactors_20200807.xlsx'
filename_GHG_emissions = 'GHG_emissions.csv'

# Filenames of output files (used as model input)
filename_buses = 'buses.csv'
filename_buses_2030 = 'buses_2030.csv'

filename_linking_transformers = 'linking_transformers.csv'
filename_linking_transformers_ts = 'linking_transformers_ts.csv'

filename_sources_fluc_res = 'sources_renewables_fluc.csv'
filename_sources_commodity = 'sources_commodity.csv'
filename_sources_commodity_2030 = 'sources_commodity_2030.csv'
filename_emission_limits = 'emission_limits.csv'
filename_sources_shortage = 'sources_shortage.csv'
filename_sources_RES = 'sources_renewables.csv'
filename_sources_RES_2030 = 'sources_renewables_2030.csv'
filename_sources_RES_ts = 'sources_renewables_ts.csv'
filename_sources_RES_ts_2030 = 'sources_renewables_ts_2030.csv'

filename_sinks_demand_el = 'sinks_demand_el.csv'
filename_sinks_demand_el_ts = 'sinks_demand_el_ts.csv'
filename_sinks_excess = 'sinks_excess.csv'

filename_transformers = 'transformers.csv'
filename_transformers_2030 = 'transformers_2030.csv'
filename_transformers_minload_ts = 'transformers_minload_ts.csv'
filename_transformers_RES = 'transformers_renewables.csv'
filename_transformers_RES_2030 = 'transformers_renewables_2030.csv'

filename_storages_el = 'storages_el.csv'
filename_storages_el_2030 = 'storages_el_2030.csv'

filename_costs_operation = 'costs_operation.csv'
filename_costs_operation_2030 = 'costs_operation_2030.csv'
filename_costs_fuel = 'costs_fuel_middle.csv'
filename_costs_fuel_2030 = 'costs_fuel_middle_2030.csv'
filename_costs_ramping = 'costs_ramping.csv'
filename_costs_ramping_2030 = 'costs_ramping_2030.csv'
filename_costs_operation_res = 'costs_operation_renewables.csv'
filename_costs_operation_res_2030 = 'costs_operation_renewables_2030.csv'
filename_costs_operation_storage = 'costs_operation_storages.csv'
filename_costs_operation_storage_2030 = 'costs_operation_storages_2030.csv'
filename_costs_carbon = 'costs_carbon.csv'
filename_costs_carbon_2030 = 'costs_carbon_2030.csv'
filename_costs_market_values = 'costs_market_values.csv'

writer = pd.ExcelWriter(path_folder_outputs+'input_data_complete.xlsx', engine='openpyxl')

## Notebook workflow settings
- The variable `year` determines, for which year the power plant status shall be evaluated. It defaults to 2017.
- Set boolean parameter which determines whether to export the complete ER power plants list for Germany.
- Define a function to transfer objects to oemof.solph's data structure.
- Introduce a boolean switch `use_new_data_set` in order to be able to quickly switch between the power plants list as of 2017 and the one as of 2019. <br>&rarr; _Note that this is only applicable for the main power plant data sources used, i. e. the OPSD list for DE(-LU) and the FRESNA powerplantmatching list for EU. The other updated data sources are basically an addition to existing ones and hence prevalently expand the data basis._
- For some plants appearing in the list(s) of power plants to be decommissioned, the decommissioning year is missing. This is defined by `shutdown_assumption`, which imposes a (single) decommissioning year for the respective plants.
- `res_capacity_projection` determines which estimate to use for RES capacity development for Germany until 2030 ("Prognos" or "EEG_2021")

In [ ]:
year = 2019
use_new_data_set = True
shutdown_assumption = 2022
cluster_transformers_DE = True
res_capacity_projection = "Prognos"  # "EEG_2021"

# Transformers

In this section, **conventional power plant** data for Germany and Europe is put together.<br>
In [oemof.solph](https://github.com/oemof/oemof-solph) conventional power plants (in the most simple approach) can be represented through (generic) so called "transformers" which have one input and one or two ouputs (two outputs for CHP).

## Data Import and initial preparation

**Main data source**:<br>
German and European Data on conventional powerplants (PPs) is taken from [Open Power System Data](https://data.open-power-system-data.org/conventional_power_plants/), downloaded on 2019-10-02:<br>
Open Power System Data. 2018. Data Package Conventional power plants. Version 2018-12-20. https://doi.org/10.25832/conventional_power_plants/2018-12-20

**UPDATE**:
A newer OPSD power plant list has been integrated, downloaded on 2021-01-04:<br>
Open Power System Data. 2020. Data Package Conventional power plants. Version 2020-10-01. https://doi.org/10.25832/conventional_power_plants/2020-10-01

Steps applied for integrating updated data set:
* Use separate reading in of data and keep older version for the sake of comparison and debugging until new data is fully integrated.
* Some columns have been renamed in the version update (old: 'country_code' &rarr; new: 'country'; old: 'fuel' &rarr; new: 'energy_source'). The old names are kept.

### German Powerplants (OPSD)

#### Read in data and do renaming
Steps applied:
- Data from conventional power plants from OPSD is read in.
- Columns not needed are dropped.
- A new country index 'DE' is assigned for all power plants, including those that are positioned in the netherlands and austria but feed into the German grid.

In [ ]:
opsd_de = pd.read_excel(path_folder_inputs+path_folder_pp_public+filename_opsd_de,
                        sheet_name='plants', index_col='id')

opsd_de.drop(
    columns=['capacity_gross_uba',
             'street', 'postcode', 'city', 'network_operator',
             'energy_source_level_1', 'energy_source_level_2', 'energy_source_level_3',
             'merge_comment', 'comment'], 
    inplace=True)

opsd_de['country'] = 'DE'
opsd_de.rename(columns={'country_code': 'country_geographical'}, inplace=True)
eu_pp_feedin_de = opsd_de[opsd_de['country_geographical'] != 'DE'].index

In [ ]:
opsd_de_new = pd.read_csv(path_folder_inputs+path_folder_pp_public+filename_opsd_de_new,
                          index_col=0)

opsd_de_new.drop(
    columns=['capacity_gross_uba',
             'street', 'postcode', 'city', 'network_operator',
             'energy_source_level_1', 'energy_source_level_2', 'energy_source_level_3',
             'merge_comment', 'comment'], 
    inplace=True)

opsd_de_new['country2'] = 'DE'
opsd_de_new.rename(columns={'country': 'country_geographical', 
                            'country2': 'country',
                            'energy_source': 'fuel'}, inplace=True)
eu_pp_feedin_de_new = opsd_de_new[opsd_de_new['country_geographical'] != 'DE'].index

if use_new_data_set:
    opsd_de_old = opsd_de.copy()
    eu_pp_feedin_de_old = eu_pp_feedin_de.copy()

    # Overwrite old OPSD data with new one
    opsd_de = opsd_de_new
    eu_pp_feedin_de = eu_pp_feedin_de_new

> _NOTE: There are two duplicated BNetzA ids in the new OPSD data set which are obtained from the original BNetzA power plants list._
> * _For the respective power plants, one block (or a share of blocks) is shutdown while the other is still operational_
> * _The shutdown power blocks' BNetzA id is manipulated to account for that circumstance._

Approach:
* Append shutdown column to index to create a MultiIndex
* Identify duplicates at index level 0 (BNetzA ID) with shutdown being not NaN.
* Get the index location
* Get list of original indices, replace the index location and reassign indices, using the string '\_SD' to indicate plants for shutdown.

In [ ]:
if use_new_data_set:
    duplicated_idx = {}
    
    opsd_multi = opsd_de.set_index('shutdown', append=True)
    
    duplicated = opsd_multi[(opsd_multi.index.get_level_values(0).duplicated(keep=False))
                             & (opsd_multi.index.get_level_values(1).notna())].index

    for el in duplicated:
        idx_el = opsd_multi.index.get_loc(el)
        duplicated_idx[el[0]] = idx_el
        
    as_list = opsd_de.index.tolist()
    for k, v in duplicated_idx.items():
        as_list[v] = k + '_SD'
    opsd_de.index = as_list

#### Prepare German conventional PP Raw Data

Steps applied:
- Exclude pumped storage, run-of-river and (other) storage technologies because they are specifically treated later on <br>&rarr; Create subsets for these technologies for later usage
- Exclude non EEG units (i.e. biomass power plants) because biomass within the EEG scheme are treated as EE source later on <br>&rarr; Create a subset for EEG technologies for later usage
- Replace fuel names from the OPSD list with the names used in _POMMES_ and rename some columns
- Assign last commissioning dates for units which had a retrofit (last commissioning date = retrofitting date)

In [ ]:
conv_de = opsd_de[(~opsd_de['technology'].isin(['Pumped storage', 'Run-of-river', 
                                                'Storage technologies', 'RES', 'Reservoir'])) &
                  (opsd_de['eeg'] == 'no')].copy()

phes_de = opsd_de.loc[opsd_de['technology'] == 'Pumped storage']
ror_de = opsd_de.loc[opsd_de['technology'] == 'Run-of-river']
batterystorage_de = opsd_de[opsd_de['technology'] == 'Storage technologies']
reservoir_de = opsd_de.loc[opsd_de['technology'].isin(['RES', 'Reservoir'])]
eeg_de = opsd_de.loc[(opsd_de['eeg'] == 'yes') & (opsd_de['technology'] != 'Run-of-river')]

In [ ]:
dict_fuels = {'Hard coal': 'hardcoal',
              'Nuclear': 'uranium',
              'Mixed fossil fuels': 'mixedfuels',
              'Lignite': 'lignite',
              'Natural gas': 'natgas',
              'Oil': 'oil',
              'Biomass and biogas': 'biomass',
              'Other fossil fuels': 'otherfossil',
              'Other fuels': 'otherfossil',
              'Waste': 'waste'}

conv_de.loc[:,'fuel'].replace(dict_fuels, inplace=True)

conv_de.rename(columns={'capacity_net_bnetza': 'capacity',
                        'eic_code_plant': 'eic_code',
                        'name_bnetza': 'name'}, inplace=True)

conv_de.loc[~conv_de['retrofit'].isna(),
            'commissioned_last'] = conv_de.loc[~conv_de['retrofit'].isna(), 'retrofit']
conv_de.loc[conv_de['retrofit'].isna(), 
            'commissioned_last'] = conv_de.loc[conv_de['retrofit'].isna(), 'commissioned']
conv_de = conv_de.astype(dtype={'commissioned_last': 'object'})

### European Powerplants (PyPSA-EUR PP matching)

European power plants data is obtained from an existing [power plant matching tool](https://github.com/FRESNA/powerplantmatching) developped by the PyPSA developpers:

F. Gotzens, H. Heinrichs, J. Hörsch, and F. Hofmann, Performing energy modelling exercises in a transparent way - The issue of data quality in power plant databases, Energy Strategy Reviews, vol. 23, pp. 1–12, Jan. 2019. <br>[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3358985.svg)](https://doi.org/10.5281/zenodo.3358985).<br>Copyright 2018-2020 Fabian Gotzens (FZ Jülich), Jonas Hörsch (KIT), Fabian Hofmann (FIAS)<br>
powerplantmatching is released as free software under the [GPLv3](http://www.gnu.org/licenses/gpl-3.0.en.html), see [LICENSE](https://github.com/FRESNA/powerplantmatching/blob/master/LICENSE) for further information.

Steps applied:
- Read in data set supplied as output of the power plant matching tool.
- Drop data for countries which are not within the scope of _POMMES_.
- Drop powerplants that feed into the German grid and are modeled in Germany.

**Update**: Use data set as of end of 2020-26-11.
* ids used are unique and consistent.
* Keep the naming from the older data set.

In [ ]:
pp_eu = pd.read_excel(path_folder_inputs+path_folder_pp_public+filename_ppmatching_eu,
                      index_col = 0)

countries_eu = {'Netherlands': 'NL', 
                'Denmark': 'DK', 
                'France': 'FR', 
                'Poland': 'PL', 
                'Switzerland': 'CH',
                'Czech Republic': 'CZ', 
                'Norway': 'NO', 
                'Sweden': 'SE', 
                'Austria': 'AT', 
                'Belgium': 'BE'}

pp_eu.drop(index=pp_eu[~pp_eu['Country'].isin(countries_eu.keys())].index, inplace=True)
pp_eu['Country'].replace(countries_eu, inplace=True)

pp_eu.drop(index=pp_eu[pp_eu['projectID'].str.contains('|'.join(eu_pp_feedin_de))].index, inplace=True)

In [ ]:
if use_new_data_set:
    pp_eu_new = pd.read_csv(path_folder_inputs+path_folder_pp_public+filename_ppmatching_eu_new,
                            index_col=0)
    
    columns_eu_old = {'DateIn': 'YearCommissioned',
                      'DateRetrofit': 'Retrofit'}
    
    pp_eu_new.rename(columns=columns_eu_old, inplace=True)
    pp_eu_new.drop(index=pp_eu_new[~pp_eu_new['Country'].isin(countries_eu.keys())].index, inplace=True)
    pp_eu_new['Country'].replace(countries_eu, inplace=True)

    pp_eu_new.drop(index=pp_eu[pp_eu['projectID'].str.contains('|'.join(eu_pp_feedin_de_new))].index, inplace=True)

In [ ]:
if use_new_data_set:
    pp_eu_old = pp_eu.copy()

    # Overwrite old power plant matching data with new one
    pp_eu = pp_eu_new

#### Bidding Zone Allocation
For Sweden and Norway shape files are used, that contain the respective bidding zones. Those plants that are not in these multipolygons are allocated based on the bidding zone of their nearest neighbour. This happens quite often for Norwegian hydro power, where the fjords are not perfectly represented by the shapes.

Shape data is obtained from the electricity map of Tomorrow. See this [GitHub issue](https://github.com/tmrowco/electricitymap-contrib/pull/1383) for Norwegian and Swedish data.<br>
Copyright (c) 2020 Tomorrow<br>
License is [MIT License](https://opensource.org/licenses/MIT).

Steps applied:
- Assign bidding zones for DK based on longitude
- Assign bidding zones for Norway and Sweden based on geometry shape data
- Assign missing bidding zone information based on smallest geodesic distance to the next bidding zone for Norway and Sweden

In [ ]:
pp_eu['bidding_zone'] = pp_eu['Country']

pp_eu.loc[pp_eu['Country'] == 'DK', 'bidding_zone'] = (
    np.where(pp_eu.loc[pp_eu['Country'] == 'DK', 'lon'] <= 10.9, 'DK1', 'DK2'))

country_dict = {'NO': 5, 'SE': 4}
for country in country_dict.keys():
    points = pp_eu.loc[(pp_eu['Country'] == country), ['lat','lon']]
    points_df = gpd.GeoDataFrame(points, 
                                 geometry=gpd.points_from_xy(points.lon, points.lat), 
                                 crs="epsg:4326")
    
    zones = pd.concat([tools.load_bidding_zone_shape(country, country + str(number),
                                                     path_folder_inputs+path_folder_marketzones) 
                       for number in range(1, country_dict[country] + 1)])
    
    points_df = gpd.sjoin(points_df, zones, how='left')
    pp_eu.loc[points_df.index, 'bidding_zone'] = points_df['id'].values
    
    for idx in pp_eu[(pp_eu['Country'] == country) & (pp_eu['bidding_zone'].isna())].index:
        lat_miss, lon_miss = pp_eu.loc[idx, ['lat', 'lon']]

        idx_min_dist = pp_eu.loc[(pp_eu['Country'] == country) &
                                   (~pp_eu['bidding_zone'].isna()), ['lat', 'lon']].apply(
            lambda x: tools.calc_dist(lat_miss, lon_miss, x.lat, x.lon), axis=1).idxmin()

        pp_eu.at[idx, 'bidding_zone'] = pp_eu.at[idx_min_dist, 'bidding_zone']

#### Prepare European conventional PP Raw Data
Steps applied:
- Rename columns and fuels such that they match the (oemof.solph) terminology used in _POMMES_
- Assign technology for hydro plants
    - _**Assumption made**: Plants under 10 kW are ROR, those above are hydro storages (reservoir)_
- Introduce subsets for some technologies (conventional and renewable ones)
- Assign last commissioning dates for units which had a retrofit (last commissioning date = retrofitting date)

In [ ]:
pp_eu.rename(
    columns={'Name': 'name', 
             'Fueltype': 'fuel', 
             'Technology': 'technology', 
             'Country': 'country',
             'Capacity': 'capacity', 
             'Efficiency': 'efficiency_el', 
             'Duration': 'duration', 
             'Volume_Mm3':'volume_m3',
             'DamHeight_m': 'damheight_m', 
             'YearCommissioned': 'commissioned', 
             'Retrofit': 'retrofit'}, 
    inplace=True)

pp_eu.loc[(pp_eu['fuel'] == 'Hydro') & (pp_eu['technology'].isna()), 'technology'] = (
    np.where(pp_eu.loc[(pp_eu['fuel'] == 'Hydro') & (pp_eu['technology'].isna()), 'capacity'] < 10,
             'Run-Of-River',
             'Reservoir'))

conv_eu = pp_eu[(~pp_eu['fuel'].isin(['Wind', 'Solar', 'Hydro']))].copy()
renewables_eu_ppmatch = pp_eu[pp_eu['fuel'].isin(['Wind', 'Solar'])]
pumpedstorage_eu = pp_eu.loc[pp_eu['technology'] == 'Pumped Storage']
ror_eu = pp_eu.loc[pp_eu['technology'] == 'Run-Of-River']
reservoir_eu = pp_eu.loc[pp_eu['technology'] == 'Reservoir']

In [ ]:
conv_eu['fuel'].replace(
    {'Hard Coal': 'hardcoal',
     'Nuclear': 'uranium',
     'Lignite': 'lignite',
     'Natural Gas': 'natgas',
     'Oil': 'oil',
     'Bioenergy': 'biomass',
     'Other': 'otherfossil'}, 
    inplace=True)

conv_eu.loc[~conv_eu['retrofit'].isna(), 
            'commissioned_last'] = conv_eu.loc[~conv_eu['retrofit'].isna(), 'retrofit']
conv_eu.loc[conv_eu['retrofit'].isna(), 
            'commissioned_last'] = conv_eu.loc[conv_eu['retrofit'].isna(), 'commissioned']

## Add projection for near future power plant development

The future estimate for power plants in 2030 for Germany is depicted using the following sources
* NEP power plant list: ÜNB / BNetzA (2019): Kraftwerksliste zum ÜNB Entwurf des Szenariorahmens zum NEP 2030,  https://www.netzentwicklungsplan.de/sites/default/files/paragraphs-files/Kraftwerksliste_%C3%9CNB_Entwurf_Szenariorahmen_2030_V2019_2_0_0.pdf, downloaded on 2019-10-22.
* Plans for commissioning of new plants put together by Julien Faist (JF). Primary data is combined from
    * UBA (2019): Genehmigte oder im Genehmigungsverfahren befindliche Kraftwerksprojekte in Deutschland, as of 01/2019, https://www.umweltbundesamt.de/sites/default/files/medien/384/bilder/dateien/4_tab_genehmigte-in_genehmigung-kraftwerksprojekte_2019-04-04.pdf, accessed 03.11.2020.
    * BDEW (2019): BDEW-Kraftwerksliste. In Bau oder Planung befindliche Anlagen ab 20 Megawatt (MW) Leistung, Anlage zur BDEW-Presseinformation vom 1.April 2019 zur Hannover Messe, https://www.bdew.de/media/documents/PI_20190401_BDEW-Kraftwerksliste.pdf, accessed 03.11.2020.
    * BNetzA (2019): Kraftwerksliste Bundesnetzagentur zum erwarteten Zu- und Rückbau 2019 bis 2022, https://www.bundesnetzagentur.de/DE/Sachgebiete/ElektrizitaetundGas/Unternehmen_Institutionen/Versorgungssicherheit/Erzeugungskapazitaeten/Kraftwerksliste/kraftwerksliste-node.html, as of 07.03.2019, accessed 03.12.2019.
    * company specific sources given in the table itself.
* The power plant 'Datteln 4' has been taken into operation meanwhile. It is obtained from a data set on exogeneous decommissioning plans which has been put together by Julien Faist as well.

The future capacity projections for Europe in 2030 are derived from the latest version of the TYNDP from ENTSO-E:<br>
ENTSO-E (2019): ENTSOE Scenario 2018 Generation Capacities, https://tyndp.entsoe.eu/maps-data, accessed 03.11.2020.

Updated sources:
* BNetzA (2020): Kraftwerksliste Bundesnetzagentur zum erwarteten Zu- und Rückbau 2019 bis 2022, https://www.bundesnetzagentur.de/DE/Sachgebiete/ElektrizitaetundGas/Unternehmen_Institutionen/Versorgungssicherheit/Erzeugungskapazitaeten/Kraftwerksliste/kraftwerksliste-node.html, as of 01.04.2020, accessed 05.01.2021.

### Determine new commissionings for Germany
Steps applied:
* Read in and use the NEP power plant data set.
    * Replace values with unknown commissioning date in NEP list. &rarr; These are planned power plants for which no commissioning date is given; it is (arbitrarily) set to 2030.
    * Filter out newly built power plants which are commissioned between 2019 and 2030.
    * Drop power to heat units since only generation the demand-side is exogeneously fixed in _POMMES_ except for electrical demand response.
    * Rename columns that shall be kept so that they match the conventional data set from OPSD.
    * Create a new data set containing only the new built power plants.
* Combine information on new-built power plants from NEP with the commissionings list put together by JF and the new commissionings information from BNetzA.
    * Read in the commissionings lists (JF's list and BNetzA list).
    * Do a pairwhise string comparison of the power plants names in order to identify duplicates, i.e. power plants that are listed in two power plants lists. This is necessary for mapping because the commissionings lists in contrary to the other lists does not have the BNA_Id as an index.
    * If there are duplicates, assign the missing capacity information from the commissionings lists data set but use the NEP data set for the remainder. The priority of the data sources is as follows:
        1. BNetzA list since it contains the most topical information.
        2. NEP list since it is one single consistent source.
        3. JF's list since it combines different sources.
* Append information for the hard coal power plant 'Datteln 4' from the hardcoal shutdown list put together by JF.
* The planned power plant Stade-Bützfleth must not be build due to the German coal phase out law (Kohleverstromungsbeendigungsgesetz - KVBG) forbidding new installations of coal-fired power plants.

In [ ]:
nep = pd.read_excel(path_folder_inputs+path_folder_pp_public+filename_nep_de)
nep.drop_duplicates(subset='BNetzA-ID', inplace=True)
nep.set_index('BNetzA-ID', inplace=True)

nep.loc[:,'Inbetriebnahme (Jahr)'].replace({'Jahr unbestimmt': 2030}, inplace=True)
nep_new = nep.loc[nep['Inbetriebnahme (Jahr)'].isin(range(2019, 2031))]

dict_nep_col_names = {'Kraftwerksname': 'name',
                      'Betreiber': 'company',
                      'Bundesland': 'state',
                      'Energieträger': 'fuel',
                      'Wärmeauskopplung KWK (Ja/Nein)': 'NEP_chp_bool',
                      'Annahmen ÜNB:\nIndustriekraftwerk (Ja/Nein)': 'NEP_ipp_bool',
                      'Inbetriebnahme (Jahr)': 'commissioned',
                      'Nettonennleistung B2035 [MW]': 'capacity'}
cols_to_drop = [col for col in nep_new.columns if col not in dict_nep_col_names.keys()]
nep_new = nep_new.rename(columns=dict_nep_col_names).drop(cols_to_drop, axis=1)

empty_df_new = pd.DataFrame(columns=conv_de.columns)
conv_de_new = pd.concat([empty_df_new, nep_new], axis=0)

conv_de_new = conv_de_new[~(conv_de_new.name.str.contains('Power to heat', 
                            flags=IGNORECASE, regex=True))]

In [ ]:
comm_conv_de = pd.read_excel(path_folder_inputs+path_folder_pp_er+filename_comm,
                             sheet_name='exo_com_transformers', 
                             index_col='new_built_id',
                             usecols="A:P")

# Exclude hard coal from data set and drop line following column headers (np.nan)
comm_conv_de = comm_conv_de[comm_conv_de['Primärenergie-Basis'] != 'Steinkohle'].drop(np.nan)

comm_BNetzA = pd.read_excel(path_folder_inputs + path_folder_pp_public + filename_comm_decomm_BNetzA,
                            sheet_name='Zu und Rückbau Bund Tab', skiprows=5, usecols='B:I', nrows=13)

comm_BNetzA = comm_BNetzA[comm_BNetzA['Energieträger'] != 'Steinkohle']
comm_BNetzA.loc[comm_BNetzA['Anlagenname'].isna(), 'Anlagenname'] = comm_BNetzA['Blockname']

Do the preparation for a string comparison of power plants names:
* Define a string for some elements to be excluded from the comparison, i.e. general terms, such as 'KW' for 'Kraftwerk' (German for power plants).
* Define some inclusion criteria in order not to use relevant information, i.e. 'GuD-Köln' shall be kept while 'GuD' as a general term shall be excluded.
* For each data set, the NEP data set, the commissionings list by JF and the commissionings list by BNetzA, take the 'name' column and perform a string split (i.e. separate strings by whitespaces) and store the result to a names DataFrame.
* For the names data frame, exclude general terms which are not relevant for the comparison by replacing them with nan values.

In [ ]:
exclude = r'GUD|CCPP|AG|.?KW|KWK|GK|in|[0-9]'
include = r'GUD-|HKW-'

names = conv_de_new['name'].str.split(expand=True).replace({None:np.nan})
names_comm = comm_conv_de['Kraftwerksname'].str.split(expand=True).replace({None:np.nan})
names_comm_BNetzA = comm_BNetzA['Anlagenname'].str.split(expand=True).replace({None:np.nan})

names = names.apply(lambda x: x.str.strip('()') if x.dtype == "object" else x)
names_comm = names_comm.apply(lambda x: x.str.strip('()') if x.dtype == "object" else x)
names_comm_BNetzA = names_comm_BNetzA.apply(lambda x: x.str.strip('()') if x.dtype == "object" else x)

for col in names.columns:
    cond1 = names[col].str.match(exclude, flags=IGNORECASE, na=False)
    cond2 = ~(names[col].str.match(include, flags=IGNORECASE, na=False))
    names.loc[cond1 & cond2, col] = np.nan
    
for col in names_comm.columns:
    cond1 = names_comm[col].str.match(exclude, flags=IGNORECASE, na=False)
    cond2 = ~(names_comm[col].str.match(include, flags=IGNORECASE, na=False))
    names_comm.loc[cond1 & cond2, col] = np.nan
    
for col in names_comm_BNetzA.columns:
    cond1 = names_comm_BNetzA[col].str.match(exclude, flags=IGNORECASE, na=False)
    cond2 = ~(names_comm_BNetzA[col].str.match(include, flags=IGNORECASE, na=False))
    names_comm_BNetzA.loc[cond1 & cond2, col] = np.nan

Do the actual string comparison of power plants names:
* Create sets of unique values from all name DataFrames containing all parts of the power plants names except for the general terms that have been excluded above.
* Create the pairwhise intersection of two sets and remove the nan value.
* Combine the relevant matches to a regex in order to use this for filtering.
* Filter out plants which are contained in two data sets and store them in separate DataFrames.
* Do a manual correction: For the term 'Kessel', the matching is not perfectly well-engineered. &rarr; I.e. 'Kessel 13' in Flensburg is detected as a match, though there is only a match for 'Kessel 7' in Köln.

In [ ]:
names_set = set(names.values.flatten())
names_comm_set = set(names_comm.values.flatten())
names_comm_BNetzA_set = set(names_comm_BNetzA.values.flatten())

# union1: compare JF's list and NEP data
union1 = names_comm_set.intersection(names_set)
union1.remove(np.nan)
filter_plants1 = '|'.join(union1)

# union2: compare BNetzA and NEP data
union2 = names_comm_BNetzA_set.intersection(names_set)
union2.remove(np.nan)
filter_plants2 = '|'.join(union2)

# union3: compare JF's list and BNetzA data
union3 = names_comm_BNetzA_set.intersection(names_comm_set)
union3.remove(np.nan)
filter_plants3 = '|'.join(union3)

# Filter out the plants which are contained in two of the data sets
duplicated_comm1 = comm_conv_de[comm_conv_de['Kraftwerksname'].str.contains(filter_plants1)]
duplicated_conv1 = conv_de_new[conv_de_new['name'].str.contains(filter_plants1)]

duplicated_comm_BNetzA2 = comm_BNetzA[comm_BNetzA['Anlagenname'].str.contains(filter_plants2)]
duplicated_conv2 = conv_de_new[conv_de_new['name'].str.contains(filter_plants2)]

duplicated_comm_BNetzA3 = comm_BNetzA[comm_BNetzA['Anlagenname'].str.contains(filter_plants3)]
duplicated_comm3 = comm_conv_de[comm_conv_de['Kraftwerksname'].str.contains(filter_plants3)]

# Do correction: Power plant name matching doesn't cover all possibilities
duplicated_comm1 = duplicated_comm1[~(duplicated_comm1['Kraftwerksname'] == 'Kessel 13')]
duplicated_comm3 = duplicated_comm3[~(duplicated_comm3['Kraftwerksname'] == 'Kessel 13')]

Do a capacity matching and assign missing capacity information:
* Assign missing capacity information from commissionings lists but use NEP power plants list as a default &rarr; i.e. drop the duplicates from the commissionings list in order to be able to combine the data sources
* Check whether power plants capacities are (nearly) the same, i.e. deviate less than 20%, in addition to the plants names being very similar. <br>
&rarr; _NOTE: This matching criterion is ignored for now, but might be integrated later. The respective results are stored in a dictionary mapping the indices to each other and containing a boolean indicating a capacity mathc or not._
* Assign missing capacity values to NEP data set.<br>
&rarr; _NOTE: The 1.2 GW capacity limit mentioned in the NEP data set is violated, but plants within the data set are not build and operated by TSOs but by third parties. Hence, they do not really affect the 1.2 GW capacity limit anyway. For more detailled information on the energy regulatory background, see the [following section](#Assign-missing-values-for-new-commissioned-plants)._

> _NOTE: So far, this is not fully integrated and done for the comparison of JF's list and the NEP power plants list only. This seems sufficient, since so far the routine is only used for assigning missing capacity values which can be achieved._

In [ ]:
# idx_mapping: key is the search string
# values are indices in data sets and info on whether or not there is a capacity match
idx_mapping1 = {}

for el in union1:
    # Address inaccuracy for string 'Kessel'
    if el == 'Kessel':
        el = 'Kessel 7'
        
    idx1 = list(comm_conv_de.loc[comm_conv_de['Kraftwerksname'].str.contains(el)].index)
    idx2 = list(conv_de_new.loc[conv_de_new['name'].str.contains(el)].index)
    
    cap1 = comm_conv_de.loc[comm_conv_de['Kraftwerksname'].str.contains(el), 
                            'elektrische Nettoleistung (MW)'].values
    cap2 = conv_de_new.loc[conv_de_new['name'].str.contains(el), 
                           'capacity'].values

    if len(cap1) > 1:
        cap1 = sum(cap1)
    else:
        cap1 = cap1[0]
    if len(cap2) > 1:
        cap2 = sum(cap2)
    else:
        cap2 = cap2[0]
    
    # Determine whether capacity is nearly the same
    match = True
    if not isinstance(cap1, str):
        interval = [cap1 * 0.8, cap1 * 1.2]
        if not isinstance(cap2, str):
            match = (cap2 >= interval[0]) & (cap2 <= interval[1])
        else:
            cap2 = cap1
            conv_de_new.loc[idx2, 'capacity'] = cap1

    idx_mapping1[el] = (idx1, idx2, match)

Adjust BNetzA commissionings list:
* Add CHP information from JF's list.
* Add IPP information based on company names.
* Add unique IDs for the plants from the BNetzA list, building up on the scheme JF introduced.
* Set CHP and IPP information for known industry power plants.

In [ ]:
comm_BNetzA[['KWK (falls bekannt)', 'IPP']] = 'nein'
for el in union3:
    # Address inaccuracy for string 'Kessel'
    if el == 'Kessel':
        el = 'Kessel 7'
    
    comm_BNetzA.loc[comm_BNetzA['Anlagenname'].str.contains(el), 
                    ['KWK (falls bekannt)', 'IPP', 'Fernwärme Leistung (MW)']] = (
        comm_conv_de.loc[comm_conv_de['Kraftwerksname'].str.contains(el)
                         & ~comm_conv_de['Kraftwerksname'].str.contains('West\n'),
                         ['KWK (falls bekannt)', 'IPP', 'Fernwärme Leistung (MW)']].values)
    comm_BNetzA.rename(index={
        k: comm_conv_de.loc[
               comm_conv_de['Kraftwerksname'].str.contains(el)
               & ~comm_conv_de['Kraftwerksname'].str.contains('West\n')].index.values[0]
           for k in comm_BNetzA.loc[comm_BNetzA['Anlagenname'].str.contains(el)].index}, 
                       inplace=True)
    
last_idx_comm = int(comm_conv_de.index.values[-1].split('_')[-1])

keys_BNetzA = [el for el in comm_BNetzA.index if isinstance(el, int)]
values_BNetzA = ['new_built_' + '{:03d}'.format(el) 
                 for el in range(last_idx_comm + 1, last_idx_comm + 1 + len(keys_BNetzA))]
comm_BNetzA_newidx = dict(zip(keys_BNetzA, values_BNetzA))
comm_BNetzA.rename(index=comm_BNetzA_newidx, inplace=True)

comm_BNetzA.loc[comm_BNetzA['Anlagenname'].str.contains('HKW'), 'KWK (falls bekannt)'] = 'yes'
comm_BNetzA.loc[comm_BNetzA['Unternehmen'].str.contains("|".join('Volkswagen, Ineos')), 'IPP'] = 'yes'

Combine the different data sets after the string matching of power plants names:
* Drop the duplicates from the commissionings lists
* Adjust the column names and some entries for consistency
* Combine the commissionings lists with the new-built power plants obtained from the NEP power plants list

In [ ]:
comm_conv_de.drop(duplicated_comm1.index.union(duplicated_comm3.index), inplace=True)
conv_de_new.drop(duplicated_conv2.index, inplace=True)

dict_comm_col_names = {
    'Kraftwerksname': 'name',
    'Unternehmen': 'company',
    'Primärenergie-Basis': 'fuel',
    'KWK (falls bekannt)': 'NEP_chp_bool',
    'IPP': 'NEP_ipp_bool',
    'Fernwärme Leistung (MW)': 'chp_capacity_uba',
    'Anlagenart': 'technology',
    'geplante Inbetriebnahme': 'commissioned',
    'elektrische Nettoleistung (MW)': 'capacity'}

dict_comm_BNetzA_names = {
    'Anlagenname': 'name',
    'Unternehmen': 'company',
    'Energieträger': 'fuel',
    'KWK (falls bekannt)': 'NEP_chp_bool',
    'IPP': 'NEP_ipp_bool',
    'Fernwärme Leistung (MW)': 'chp_capacity_uba',
    'Voraussichtliche Aufnahme der kommerziellen Strom-einspeisung': 'commissioned',
    'Geplante Netto-Nennleistung (elektrisch) der Investition in MW (Pumpspeicher: Turbinenbetrieb)': 'capacity'}

cols_to_drop = [col for col in comm_conv_de.columns 
                if col not in dict_comm_col_names.keys()]
cols_to_drop_BNetzA = [col for col in comm_BNetzA.columns 
                       if col not in dict_comm_BNetzA_names.keys()]

comm_conv_de = comm_conv_de.rename(columns=dict_comm_col_names).drop(cols_to_drop, axis=1)
comm_BNetzA = comm_BNetzA.rename(columns=dict_comm_BNetzA_names).drop(cols_to_drop_BNetzA, axis=1)

# Replace CHP values in order to be able to use the same data prep steps
comm_conv_de.loc[:, ['NEP_chp_bool', 'NEP_ipp_bool']].replace({'ja': 'Ja', 'nein': 'Nein'}, inplace=True)
comm_BNetzA.loc[:, ['NEP_chp_bool', 'NEP_ipp_bool']].replace({'ja': 'Ja', 'nein': 'Nein'}, inplace=True)

conv_de_new = pd.concat([conv_de_new, comm_conv_de, comm_BNetzA], axis=0)

Append information on individual plants ('Datteln 4' & 'HKW Dieselstr.'):
* Read in decommissioning data set for hardcoal
* Access plant Datteln 4 and do some renaming
* Append it to the new-built power plants data set for Germany
* Assign CHP information (based on information from the plant operator Uniper: https://www.uniper.energy/de/datteln-4, accessed 03.11.2020)
* Add missing capacity information for HKW Dieselstr. Halle from plant operators website:
https://evh.de/privatkunden/unternehmen/energiepark/dieselstra%C3%9Fe, accessed 03.11.2020

In [ ]:
decomm_hardcoal_de = pd.read_excel(path_folder_inputs+path_folder_pp_er+filename_decomm_coal,
                                   sheet_name='Kohlekommission SK Rückb. 2035', 
                                   index_col='Kraftwerksnummer Bundesnetzagentur',
                                   usecols="A:M")

decomm_hardcoal_col_names = {
    'Unternehmen': 'company',
    'Kraftwerksname': 'name',
    'Energieträger': 'fuel',
    'Aufnahme der kommerziellen Stromerzeugung der derzeit in Betrieb befindlichen Erzeugungseinheit\n(Datum/Jahr)':
    'commissioned',
    'Geplante Ausserbetriebnahme': 'shutdown',
    'Netto-Nennleistung (elektrische Wirkleistung) in MW': 'capacity',
}

datteln4 = decomm_hardcoal_de[decomm_hardcoal_de['Kraftwerksname'] == 'Datteln 4'].rename(
    columns=decomm_hardcoal_col_names).drop(
    columns=[col for col in decomm_hardcoal_de.columns if col not in decomm_hardcoal_col_names.keys()])

conv_de_new = conv_de_new.append(datteln4)
conv_de_new.loc[conv_de_new['name'] == 'Datteln 4', 
                ['NEP_chp_bool', 'NEP_ipp_bool', 'chp_capacity_uba']] = ('Ja', 'Nein', 380)

conv_de_new.loc[conv_de_new['name'] == 'HKW Dieselstr.\n(Modernisierung)', ['capacity', 'chp_capacity_uba']] = (47, 34)

### Assign missing values for new commissioned plants

* Assign new values / missing values in the data set containing new plants:
    * energy sources &rarr; Use names consistent with OPSD naming conventions
    * country &rarr; Assign Germany (DE) for all plants
    * status &rarr; Set status to operating (for 2030 consideration)
    * eeg status &rarr; Assign no, i.e. no RES power plants under the German Renewable Energies Act (EEG)
    * commissioning year &rarr; commissioned_last is the same as initial commissioning year
    * technology &rarr; See below for the logic used
    * capacity &rarr; Has to be distributed for some units (see below)

* Split data sets for power plants and pumped hydro as well as run of river plants since they are treated separately.

In [ ]:
conv_de_new.loc[:,'fuel'].replace(
    {'Pumpspeicher': 'Pumped storage',
     'Erdgas': 'natgas',
     'Kuppelgas': 'otherfossil',
     'Sonstige': 'otherfossil',
     'Mineralölprodukte': 'oil',
     'Wasser': 'Run-of-river',
     'Biomasse': 'biomass',
     'Steinkohle': 'hardcoal',
     'Mehrere Energieträger': 'natgas',
     'Sonstige Energieträger\n(nicht erneuerbar)': 'otherfossil'},
    inplace=True)

conv_de_new.loc[:,['country_geographical', 'country']] = 'DE'
conv_de_new['status'] = 'operating'
conv_de_new['eeg'] = 'no'
conv_de_new['commissioned_last'] = conv_de_new['commissioned']

Technology is added according to the following logic:
* If technology information does exist (new-built plants from the UBA list), assign a name that is consistent with the convention used here later on
* If plant is a gas-fired combined cycle plant &rarr; Assign combined cycle (CC)
* Else if plant is gas-fired &rarr; Assing gas turbine (GT)
* Else &rarr; assign steam turbine (ST) for all remaining plants

For some plants, no capacity values are given, instead a message _'be aware of 1.2 GW capacity limit'_ is stated.

Energy regulatory background:
* § 13k EnWG formerly allowed TSOs to build and operate gas-fired back up power plants up to an overall capacity of 2 GW resp. up to the amount of capacity which the German regulatory body (BNetzA) assumed to be necessary.
* The BNetzA determined a maximum amount of 1.2 GW in 2017, see: https://www.bundesnetzagentur.de/SharedDocs/Downloads/DE/Sachgebiete/Energie/Unternehmen_Institutionen/Versorgungssicherheit/Berichte_Fallanalysen/BNetzA_Netzstabilitaetsanlagen13k.pdf?__blob=publicationFile&v=3 (accessed 22.10.2020)
* These 1.2 GW served as a basis for the NEP 2030 (version 2019, 2nd draft) which is used here.
* § 13k does no longer exist and is substituted by § 11 (3) EnWG which explicitly forbids TSOs to build and operate power plants by their own which would be not in line with the unbundling regulations.

Approach applied here: The overall capacity of 1.2 GW for new-built gas power plants is equally split between the different plants.

In [ ]:
conv_de_new.loc[:, 'technology'].replace(
    {'GuD': 'CC',
     'GM': 'M',
     'G/AK': 'CC',
     'BGT': 'GT',
     'BST': 'ST'}, 
    inplace=True)

conditions = [(conv_de_new.name.str.contains('GUD|CCPP', 
                                          flags=IGNORECASE, 
                                          regex=True)) & (conv_de_new.fuel == 'natgas'),
             (~conv_de_new.name.str.contains('GUD|CCPP', 
                                          flags=IGNORECASE, 
                                          regex=True)) & (conv_de_new.fuel == 'natgas')]
choices = ['CC', 'GT']

conv_de_new['technology_temp'] = np.select(conditions, choices, default='ST')

conv_de_new.loc[conv_de_new['technology'].isna(), 'technology'] = conv_de_new['technology_temp']
conv_de_new.drop(columns=['technology_temp'], inplace=True)

amount = conv_de_new.loc[conv_de_new['capacity'] == '1,2-GW-Beschränkung beachten!'].shape[0]
abs_value = round(1200 / amount)

conv_de_new.loc[conv_de_new['capacity'] == '1,2-GW-Beschränkung beachten!', 
                'capacity'] = abs_value

conv_de_new['capacity'] = conv_de_new['capacity'].astype(float)

phes_de_new = conv_de_new.loc[conv_de_new['fuel'] == 'Pumped storage']
ror_de_new = conv_de_new.loc[conv_de_new['fuel'] == 'Run-of-river']
conv_de_new = conv_de_new.loc[~(conv_de_new.index.isin(phes_de_new.index.append(ror_de_new.index)))]

### European power plant development

For the European power plant park development, the TYNDP data is used.

Steps applied:
* Read in the TYNDP generation data (aggregated at the level of bidding zones):
    * Use scenario data from scenario _Distributed Generation_ which has shares of 51% RES for electricity and 3.6% RES for gas in 2030
    * Filter out the countries for closer consideration (using a regular expression)
* Aggeregate capacities for country data set they belong to (for DK and FR)
* Adjust naming of bidding zones to be consistent with the one used here
* Rename column names such that they match the names for the energy carriers used here
* Distribute aggregated data for Norway across the different bidding zones
    * Capacities for Norway are aggregated for the bidding zones NO1, NO2 and NO5, but capacity information is needed at the level of individual bidding zones.
    * As an assumption, capacities for the bidding zones are distributed in the same manner as capacities are distributed in the status quo. Therefore, the capacity shares in the status quo are determined in the first place.
* Split the data set into conventional and RES as well as ROR data and do some renaming.

In [ ]:
pp_eu_tyndp = pd.read_excel(path_folder_inputs+path_folder_pp_public+filename_tyndp_eu,
                            sheet_name = '2030 DG',
                            skiprows=2,
                            index_col = 0)

pp_eu_tyndp = pp_eu_tyndp.filter(regex="|".join(list(countries_eu.values())), axis=0)

agg_list = {'DKw': ['DKw', 'DKKF'],
            'FR': ['FR', 'FR15']}

for k, v in agg_list.items():
    pp_eu_tyndp.loc[k] = pp_eu_tyndp.loc[v[0]].add(pp_eu_tyndp.loc[v[1]])
    pp_eu_tyndp.drop(v[1], inplace=True)

pp_eu_tyndp.rename(
    index={'DKe': 'DK2',
           'DKw': 'DK1',
           'NOs': 'NO1+NO2+NO5',
           'NOm': 'NO3',
           'NOn': 'NO4'},
    inplace=True)
 
pp_eu_tyndp.rename(
    columns={'Biofuels': 'biomass',
             'Gas': 'natgas',
             'Hard coal': 'hardcoal',
             'Hydro-pump': 'PHES_capacity_pump',
             'Hydro-run': 'PHES_capacity',
             'Hydro-turbine': 'PHES_capacity_turbine',
             'Lignite': 'lignite',
             'Nuclear': 'uranium',
             'Oil': 'oil',
             'Othernon-RES': 'otherfossil',
             'Other RES': 'otherRES',
             'Solar-thermal': 'solarthermal',
             'Solar-\nPV': 'solarPV',
             'Wind-\non-shore': 'windonshore',
             'Wind-\noff-shore': 'windoffshore'},
    inplace=True)

NO_dict = {}
NO_zones = ['NO1', 'NO2', 'NO5']

for country in NO_zones:
    NO_dict[country] = conv_eu[conv_eu['bidding_zone'] == country].capacity.sum()
NO_cap = sum(NO_dict.values())

NO_cap_shares_dict = {}
for k, v in NO_dict.items():
    NO_cap_shares_dict[k] = float(v / NO_cap)
    try:
        pp_eu_tyndp.loc[k] = pp_eu_tyndp.loc['+'.join(NO_zones)].copy() * NO_cap_shares_dict[k]
    except KeyError:
        pass
    
pp_eu_tyndp.drop(index=['NO1', '+'.join(NO_zones)], inplace=True)

pp_eu_tyndp.reset_index(inplace=True)

RES_to_sep = ['solarPV', 'windonshore', 'windoffshore']
Hydro_to_sep = ['PHES_capacity_pump', 'PHES_capacity', 'PHES_capacity_turbine']

pp_eu_2030 = pd.melt(pp_eu_tyndp, id_vars=['Country/Installed capacity (MW)'])

pp_eu_2030 = pp_eu_2030[pp_eu_2030['value'] != 0].rename(
    columns={'Country/Installed capacity (MW)': 'country',
             'variable': 'fuel',
             'value': 'capacity'})

conv_eu_2030 = pp_eu_2030[(~pp_eu_2030['fuel'].isin(RES_to_sep + Hydro_to_sep))].copy()
renewables_eu_2030 = pp_eu_2030[pp_eu_2030['fuel'].isin(RES_to_sep)]
pumpedstorage_eu_2030 = pp_eu_2030.loc[pp_eu_2030['fuel'].isin(Hydro_to_sep)]

## Assign technology

- Steps applied for Germany
    - Use data with results from manual research / matching of department (ER_man)<br>
      _Caution: Data set contains duplicates &rarr; take the first occurence and drop the rest_
    - Fill missing technology data from OPSD
    
- Steps applied for Europe
    - Take technology data from OPSD
    - Further missing values are assigned by energy carrier in the following way:<br>
      natgas &rarr; GT<br>
      rest &rarr; ST
    
- Additional steps taken for both
    - Rename original technology name
    - If plants technology is 'CC' and fuel is not in natgas or oil, 'ST' is assigned instead
    - A technology and fuel combination is specified in order to assign efficiencies on this basis.

In [ ]:
techs = pd.read_excel(path_folder_inputs+path_folder_pp_er+filename_techs_de)
dict_techs_ER_man = {'T': 'GT', 'SPP': 'ST'}
techs = techs.drop_duplicates(subset='bnaID').set_index('bnaID').replace(dict_techs_ER_man)

dict_techs_ospd = {'Steam turbine': 'ST', 
                   'Gas turbine': 'GT', 
                   'Combined cycle': 'CC', 
                   'Combustion Engine': 'M'}

conv_de['technology'].replace(dict_techs_ospd, inplace=True)

conv_de = conv_de.join(techs['ERman'], how='left')
conv_de.loc[~conv_de['ERman'].isna(), 'technology'] = conv_de.loc[~conv_de['ERman'].isna(), 'ERman']
conv_de.drop(columns=['ERman'], inplace=True)

conv_de.loc[(conv_de['technology'] == 'CC')
            & (~conv_de['fuel'].isin(['natgas', 'oil'])), 'technology'] = 'ST'

conv_de['tech_fuel'] = conv_de['technology'] + '_' + conv_de['fuel']

dict_techs_ppmatch = {'Steam Turbine': 'ST', 
                      'OCGT': 'GT', 
                      'CCGT': 'CC'}

conv_eu['technology'].replace(dict_techs_ppmatch, inplace=True)

dict_tech_eu = {'natgas': 'GT', 
                'hardcoal': 'ST', 
                'uranium': 'ST', 
                'otherfossil': 'ST',
                'oil': 'ST', # efficiency later assigned on is worse than for GT
                'lignite': 'ST', 
                'biomass': 'ST'}

conv_eu.loc[conv_eu['technology'].isna(), 'technology'] = (
    conv_eu.loc[conv_eu['technology'].isna(), 'fuel'].replace(dict_tech_eu))

conv_eu.loc[(conv_eu['technology'] == 'CC')
            & (~conv_eu['fuel'].isin(['natgas', 'oil'])), 'technology'] = 'ST'

conv_eu['tech_fuel'] = conv_eu['technology'] + '_' + conv_eu['fuel']

conv_de_new['tech_fuel'] = conv_de_new['technology'] + '_' + conv_de_new['fuel']

## Assign electrical efficiencies

Data sources used and steps applied:
- Basic data source is the given electrical efficiency information of OPSD
- Missing values for efficiency are filled based on manual research by ER department. Several data sources are combined in the following priority order:
    - manual researched efficiencies by Robin Claus (RC), 18.09.2018
    - manual researched efficiencies from master thesis of Daniel Peschel (DP), 04.03.2016
- Remaining missing values are filled by a linear regression approach from [DIW Data Documentation No. 72 from 2014](https://www.diw.de/documents/publikationen/73/diw_01.c.440963.de/diw_datadoc_2014-072.pdf) (see citation below)
- For power plants with a latest commissioning date before 1950, the electrical efficiency value for the respective fuel / technology combination of 1990 is (arbitrarily) chosen.
* For oil plants, a minimum efficiency of 30% is introduced.

Egerer, Jonas, Gerbaulet, Clemens, Ihlenburg, Richard, Kunz, Friedrich, Reinhard, Benjamin, Hirschhausen, Christian von, Weber, Alexander, Weibezahn, Jens (2014): Electricity Sector Data for Policy-Relevant Modeling: Data Documentation and Applications to the German and European Electricity Markets, DIW and TU Berlin, WIP, DIW Data Documentation 72, Berlin, March 2014. © DIW Berlin, 2014.

DIW regression / interpolation:
- The efficiencies are based on the "assumption table" which includes the electrical efficiency information based on linear regressions by DIW.<br>
- It includes values for: uranium (U/ST), lignite (L/ST), hard coal (H/ST), natural gas (NG/GT), oil (O/CB)
- All other values were derived from research by Leticia Encinas Rosa (LER) and are time invariant (own assumptions)
    
For new-built power plants, the efficiency estimates from DIW are projected into the future.

In [ ]:
# Extract data from manual research
eff_RC = pd.read_excel(path_folder_inputs+path_folder_pp_er+filename_eff_de, 
                       index_col='bnaID').rename(
    columns={'eta_el': 'efficiency_el_RC'})
conv_de = conv_de.join(eff_RC['efficiency_el_RC'])

chp_DP = pd.read_excel(path_folder_inputs+path_folder_pp_er+filename_peschel, 
                       index_col='ID')
chp_DP['efficiency_el_DP'] = chp_DP['Wirkungsgrad'].replace(0, np.nan)*0.01
conv_de = conv_de.join(chp_DP['efficiency_el_DP'])

conv_de.rename(columns = {'efficiency_data': 'efficiency_el_OPSD'}, inplace=True)

conv_de['efficiency_el'] = np.nan

# Fill the data gaps
dict_eff_ordinv = ['efficiency_el_RC', 'efficiency_el_DP', 'efficiency_el_OPSD']
for eff in dict_eff_ordinv:
    conv_de.loc[(conv_de['efficiency_el'].isna()) & (~conv_de[eff].isna()), 'efficiency_el'] = (
        conv_de.loc[(conv_de['efficiency_el'].isna()) & (~conv_de[eff].isna()), eff])

In [ ]:
# Use DIW interpolation for efficiencies
assumptions_eff_el = pd.read_excel(path_folder_inputs+path_folder_assumptions+filename_assumptions,
                                   sheet_name='efficiencies', index_col=0)

arr = conv_de.loc[conv_de['efficiency_el'].isna(), ['commissioned_last', 'tech_fuel']].to_numpy()
conv_de.loc[conv_de['efficiency_el'].isna(), 'efficiency_el'] = (
    [tools.assign_eff_el_interpol(yr, tf, assumptions_eff_el) 
     for yr, tf in arr])

# Existing plants
arr = conv_eu.loc[conv_eu['efficiency_el'].isna(), ['commissioned_last', 'tech_fuel']].to_numpy()
conv_eu.loc[conv_eu['efficiency_el'].isna(), 'efficiency_el'] = (
    [tools.assign_eff_el_interpol(yr, tf, assumptions_eff_el) 
     for yr, tf in arr])

conv_de.drop(columns=['efficiency_el_OPSD', 'efficiency_source', 'efficiency_estimate',
                      'efficiency_el_RC', 'efficiency_el_DP'], inplace=True)

conv_de.loc[(conv_de['fuel'] == 'oil') & (conv_de['efficiency_el'] < 0.3), 'efficiency_el'] = 0.3

# New built plants
arr = conv_de_new.loc[:, ['commissioned_last', 'tech_fuel']].to_numpy()
conv_de_new.loc[:, 'efficiency_el'] = (
    [tools.assign_eff_el_interpol(yr, tf, assumptions_eff_el) 
     for yr, tf in arr])

conv_de_new.drop(columns=['efficiency_data', 'efficiency_source', 'efficiency_estimate'], 
                 inplace=True)

## Assign CHP information

Data sources used and steps applied:
- The following CHP categories are distincted:
    - industrial power plants (IPP)
    - district heating power plants (CHP) and
    - power plants that are dispatched solely on a electricity market basis, i.e. that do not have a minimum load profile to serve heat demand (EMB)
- The following sources are evaluated in order to obtain information on CHP eligibility and type:
    - OPSD power plant list (information if a plant uses CHP is from BNetzA; the type, i.e., IPP, CHP (district heating) is from UBA)
    - NEP 2030 power plant list
    - manual research information from master thesis of DP, 04.03.2016

> _Note: CHP information is used in POMMES to assign minimum load values resp. profiles for these plants.<br>Heat usage in turn is out of the current scope of POMMES_ 

In [ ]:
if use_new_data_set:
    # Add missing IPP information for new power plants in OPSD list
    companies = '|'.join(['BMW', 'K\+S', 'Papierfabrik'])
    diff_ix = opsd_de_new.index.difference(opsd_de_old.index)
    conv_de.loc[conv_de.index.isin(diff_ix) & 
                conv_de['company'].str.contains(companies), 'type'] = 'IPP'

In [ ]:
# from OPSD
conv_de.rename(columns={'chp': 'OPSD_chp_bool', 
                        'type': 'OPSD_chp_type'}, inplace=True)
chp_type = conv_de[['OPSD_chp_bool', 'OPSD_chp_type']].copy()
chp_type['OPSD_ipp_bool'] = np.where(chp_type['OPSD_chp_type'] == 'IPP', 'yes', 'no')

# from NEP 2030
chp_type = chp_type.join(nep[['Wärmeauskopplung KWK (Ja/Nein)', 
                              'Annahmen ÜNB:\nIndustriekraftwerk (Ja/Nein)']]).rename(
    columns={'Wärmeauskopplung KWK (Ja/Nein)': 'NEP_chp_bool',
             'Annahmen ÜNB:\nIndustriekraftwerk (Ja/Nein)': 'NEP_ipp_bool'})

chp_type_new = conv_de_new[['NEP_chp_bool', 'NEP_ipp_bool']].copy()

In [ ]:
# Existing plants
chp_type['chp_bool'] = np.where((chp_type['OPSD_chp_bool'] == 'yes') | (chp_type['NEP_chp_bool'] == 'Ja'),
                                'yes', 'no')

conditions = [(chp_type['chp_bool'] == 'yes') 
              & (chp_type['OPSD_ipp_bool'] != 'yes') & (chp_type['NEP_ipp_bool'] != 'Ja'),
              (chp_type['OPSD_ipp_bool'] == 'yes') | (chp_type['NEP_ipp_bool'] == 'Ja')]
choices = ['chp', 'ipp']

chp_type['chp_type'] = np.select(conditions, choices, default='emb')

conv_de.drop(columns=['OPSD_chp_bool', 'OPSD_chp_type'], inplace=True)
conv_de = conv_de.join(chp_type[['chp_bool', 'chp_type']], how='left').rename(columns={'chp_type': 'type'})

# New built plants
chp_type_new['chp_bool'] = np.where(chp_type_new['NEP_chp_bool'] == 'Ja',
                                    'yes', 'no')

conditions = [(chp_type_new['chp_bool'] == 'yes') & (chp_type_new['NEP_ipp_bool'] == 'Nein'), 
              chp_type_new['NEP_ipp_bool'] == 'Ja']
choices = ['chp', 'ipp']

chp_type_new['chp_type'] = np.select(conditions, choices, default='emb')

conv_de_new.drop(columns=['NEP_chp_bool', 'NEP_ipp_bool', 'type', 'chp'], inplace=True)
conv_de_new = conv_de_new.join(chp_type_new[['chp_bool', 'chp_type']], how='left').rename(
    columns={'chp_type': 'type'})

# By assumption all pps in EU are emb
conv_eu['type'] = 'emb'

## Drop information for power plants not needed

Steps applied:
- Remove plants that were shutdown prior to 2017 from the power plants list &rarr; _NOTE: 2017 is the base year used for historical backcasting. In priciple, any year up to 2018 (state of the data set) can be used here withouth affecting the future power plants data set that is created afterwards._
- Drop all information not needed for the model run in _POMMES_ (for the German and the European power plants list)

In [ ]:
# German power plants
conv_de.drop(index=conv_de[conv_de['shutdown'] < year].index, inplace=True)
conv_de.drop(index=conv_de[~((conv_de['status']=='shutdown') |
                             (conv_de['status']=='operating'))].index, inplace=True)

conv_de.drop(columns=['name', 'block_bnetza', 'name_uba', 'company', 'state', 'lat', 'lon',
                      'country_geographical',  'technology', 'chp_capacity_uba',
                      'commissioned', 'commissioned_original', 'retrofit', 'eic_code',
                      'eic_code_block', 'eeg', 'network_node', 'voltage', 'chp_bool'], inplace=True)

# European power plants
conv_eu = conv_eu[['fuel', 'capacity', 'efficiency_el', 'bidding_zone', 'tech_fuel', 'type']]
conv_eu.rename(columns={'bidding_zone': 'country'}, inplace=True)

# New built German power plants
conv_de_new.drop(columns=['name', 'block_bnetza', 'name_uba', 'company', 'state', 'lat', 'lon',
                      'country_geographical',  'technology', 'chp_capacity_uba',
                      'commissioned', 'commissioned_original', 'retrofit', 'eic_code',
                      'eic_code_block', 'eeg', 'network_node', 'voltage', 'chp_bool'], inplace=True)

## Assign commissioning and decomissioning estimates

Steps applied:
- For plants which don't have a commissioning date, assign 1990
- Calculate year when plant should be commissioned based on a lifetime calculation
- Create a table which is indexed by power plants (index) and years (columns)
- Fill in commissioning / decommissioning information:
    - If a plant is commissioned in a certain year, the respective cell contains its capacity with a positive sign
    - If a plant is decommissioned in a certain year, the respective cell contains its capacity with a negative sign

**Logic for determining decommissioning years:**
* Check if decommissioning year is prior to the starting time of the simulation run
* If so, check what type of plant is given:
    * For some plants, shutdown information has been put together by Julien Faist in a decommissionings list. Primary data sources are:
        * BNetzA (2019): Kraftwerksstilllegungsanzeigenliste, as of 01.04.2019, https://www.bundesnetzagentur.de/DE/Sachgebiete/ElektrizitaetundGas/Unternehmen_Institutionen/Versorgungssicherheit/Erzeugungskapazitaeten/KWSAL/KWSAL_node.html, accessed prior to 01.04.2020.
        * BNetzA (2019): Kraftwerksliste zum erwarteten Zu. und Rückbau 2019 bis 2022, as of 01.04.2019, https://www.bundesnetzagentur.de/DE/Sachgebiete/ElektrizitaetundGas/Unternehmen_Institutionen/Versorgungssicherheit/Erzeugungskapazitaeten/Kraftwerksliste/kraftwerksliste-node.html, accessed prior to 01.04.2020.
        * AtG: https://www.gesetze-im-internet.de/atg/, accessed 03.11.2020.
    * For coal power plans, a shutdown plan is derived from the KVBG: https://www.gesetze-im-internet.de/kvbg/, accessed 03.11.2020, which in turn is based on the recommendations made by the so-called "Kohlekommission" (Abschlussbericht der Kommission „Wachstum, Strukturwandel und Beschäftigung“ 2019). The more ambitious strategy is used, i.e. a shutdown of all power plants until 2035 already instead of 2038. The data has been put together by Julien Faist in a decommissionings list for coal power plants.
    * For the remaining plants, decommissionings occur in the x years (default: x=10) following a number of years (offset; default=5) after the start year, i.e. the one for which no OPSD data is available anymore. In each year, around 100/x% of the overall capacity to be decommissioned is actually decommissioned, whereby only entire blocks or plants are decommissioned. The order is determined by plant age and electrical efficiency.
* Else set decommissioning year to commissioning year + unit lifetime

> _NOTE:_
> _For decommissionings based on unit age, the five years after the start year are chosen since_
> * _Some plants have already exceeded their lifetime by far and_
> * _Plant operators are obliged to report shutdowns planned within the next couple of years._

Updated sources:
* BNetzA (2020): Kraftwerksstilllegungsanzeigenliste, as of 15.04.2020, https://www.bundesnetzagentur.de/DE/Sachgebiete/ElektrizitaetundGas/Unternehmen_Institutionen/Versorgungssicherheit/Erzeugungskapazitaeten/KWSAL/KWSAL_node.html, accessed 05.01.2021.
* BNetzA (2020): Kraftwerksliste Bundesnetzagentur zum erwarteten Zu- und Rückbau 2019 bis 2022, https://www.bundesnetzagentur.de/DE/Sachgebiete/ElektrizitaetundGas/Unternehmen_Institutionen/Versorgungssicherheit/Erzeugungskapazitaeten/Kraftwerksliste/kraftwerksliste-node.html, as of 01.04.2020, accessed 05.01.2021.

### Add exogeneous decommissioning info (KWSAL, AtG, other)

Steps applied:
* Include exogenous decommissioning information which has been put together by JF.
* Include exogeneous decommissioning information provided by the BNetzA from the power plants list on expected commissionings and decommissionings as well as the power plants list for reported shutdown plans (KWSAL).
* Handle duplicates in the BNetzA lists.
* Exclude system relevant power plants (which are assumed to be kept online).
* Determine the plants in the decommissioning data set which are still included in the OPSD data set.
* Assign shutdown dates either from Julien's research or from the updated BNetzA list for those plants.
* Assing decommissioning assumptions for the remaining plants:
    * Filter for plants in the KWSAL which are not system-relevant and which do neither appear in the decommissionings lists already evaluated (JF's list and the BNetzA commissionings and decommissionings list) nor in the data set of the conventional plants for Germany.
    * Assign a (single) shutdown year by assumption.

> _NOTE:_
> * _The system relevance decision has to be renewed at least every two years by the TSOS in agreement with the BNetzA. So the respective plants might be decommissioned later on._
> * _There are some plants which are not included in the conv_de dataset anymore, but still appear in the KWSAL. This is due to the fact that their status is not 'operational' anymore. Hence, these plants are already some kinds of special cases outside the EOM such as security reserve of lignite units and thus do not (explicitly) need to be considered in POMMES._
> _The remainder of plants is either dropped from the conventional data set due to its shutdown status or already covered by the other decommissionings lists._

In [ ]:
# Research by JF
decomm_conv_de = pd.read_excel(path_folder_inputs+path_folder_pp_er+filename_decomm,
                               sheet_name='Decommissioning Stand 04.2019', 
                               index_col='Kraftwerksnummer Bundesnetzagentur')

decomm_conv_de = decomm_conv_de[decomm_conv_de['Systemrelevanz'].isna()]

decomm_idx = decomm_conv_de.index.intersection(conv_de.index)

conv_de.loc[decomm_idx,'shutdown'] = decomm_conv_de.loc[decomm_idx, 'Geplante Ausserbetriebnahme']

# BNetzA list
decomm_BNetzA = pd.read_excel(path_folder_inputs + path_folder_pp_public + filename_comm_decomm_BNetzA,
                              sheet_name='Zu und Rückbau Bund Tab', skiprows=24, nrows=28, index_col=0)

trimmed_idx = decomm_BNetzA.index.str.strip('*').str.split(', ')
decomm_ids = [item for sublist in list(trimmed_idx) for item in sublist]

# Create a new entry for each power plant in BNetzA list
for idx in list(trimmed_idx):
    if len(idx) > 1:
        for subidx in idx:
            decomm_BNetzA.loc[subidx] = decomm_BNetzA.loc[decomm_BNetzA.index.str.contains(subidx)].values[0]
            
conv_de.loc[[el for el in decomm_ids 
             if el in conv_de.index], 'shutdown'] = decomm_BNetzA[
    'Voraussichtlicher Zeitpunkt der endgültigen Aufgabe (Jahr) gemäß Unternehmensplanung']

# Do some corrections
conv_de.at['BNA0969b', 'shutdown'] = decomm_BNetzA.at[
    'BNA0969b*', 
    'Voraussichtlicher Zeitpunkt der endgültigen Aufgabe (Jahr) gemäß Unternehmensplanung']

conv_de['shutdown'].replace({'2020 bis 2022': '2021',
                             '2021 bis 2023': '2022'}, inplace=True)

# KWSAL list (BDEW)
decomm_KWSAL = pd.read_csv(path_folder_inputs + path_folder_pp_public + filename_KWSAL,
                           encoding='ISO-8859-1', index_col=0, delimiter=';', decimal=",")

unique_KWSAL_idx = decomm_KWSAL.index.str.split('\n')
unique_KWSAL_id_flat = [item for sublist in unique_KWSAL_idx for item in sublist]

for idx in list(unique_KWSAL_idx):
    if len(idx) > 1:
        for subidx in idx:
            decomm_KWSAL.loc[subidx] = decomm_KWSAL.loc[decomm_KWSAL.index.str.contains(subidx)].values[0]

# Handle different approaches for HKW Wilmersdorf (blocks 2 and 3 are aggregated, while block 1 received a new index)
duplicated_KWSAL_idx = {}

decomm_KWSAL_multi = decomm_KWSAL.set_index('Kraftwerksblock', append=True)
duplicated_KWSAL = decomm_KWSAL_multi[(decomm_KWSAL_multi.index.get_level_values(0).duplicated(keep=False))
                         & (decomm_KWSAL_multi.index.get_level_values(1) == 'HKW Wilmersdorf GT 1')].index

for el in duplicated_KWSAL:
    idx_el = decomm_KWSAL_multi.index.get_loc(el)
    duplicated_KWSAL_idx[el[0]] = idx_el

as_list = decomm_KWSAL.index.tolist()
for k, v in duplicated_KWSAL_idx.items():
    as_list[v] = k + '_SD'
decomm_KWSAL.index = as_list

decomm_KWSAL = decomm_KWSAL.groupby(decomm_KWSAL.index).agg({
    'Kraftwerksbetreiber': 'first', 
    'Kraftwerksblock': sum,
    'Netto-Nennleistung in MW laut KW-Liste': sum,
    'Stilllegungsanzeigentyp': 'first',
    'Systemrelevanz von zur Stilllegung angezeigten KW-Blöcken gemäß ÜNB': 'first'})

sd_plants = decomm_KWSAL[
    ~(decomm_KWSAL['Systemrelevanz von zur Stilllegung angezeigten KW-Blöcken gemäß ÜNB'] == 'x')].index

conv_de.loc[conv_de.shutdown.isna() & 
            conv_de.index.isin(conv_de.index.intersection(sd_plants)), 'shutdown'] = shutdown_assumption

### Add exogenous decommissioning for coal (KVBG)

Steps applied:
* Include exogenous decommissioning information which has been put together by Julien Faist from the German Kohleverstromungsbeendigungsgesetz (KVBG) based on the Kohlekommission's recommendations for a coal phase out plan until 2035.
* Determine the plants from coal phase out list occuring in the conventional power plants list (from OPSD) but not in the already evaluated decommissioning list from above.

In [ ]:
# read in and combine data sets for lignite and hardcoal
decomm_lignite_de = pd.read_excel(path_folder_inputs+path_folder_pp_er+filename_decomm_coal,
                                  sheet_name='Kohlekommission BK Rückb. 2035', 
                                  index_col='Kraftwerksnummer Bundesnetzagentur',
                                  usecols="A:M")

decomm_hardcoal_de = pd.read_excel(path_folder_inputs+path_folder_pp_er+filename_decomm_coal,
                                   sheet_name='Kohlekommission SK Rückb. 2035', 
                                   index_col='Kraftwerksnummer Bundesnetzagentur',
                                   usecols="A:M")

decomm_coal_de = pd.concat([decomm_lignite_de, decomm_hardcoal_de])

decomm_coal_idx = pd.Index([el for el in decomm_coal_de.index 
                            if el in conv_de.index 
                            and el not in decomm_idx])

# Assign shutdown dates from Julien's research (possible path derived from KVBG)
conv_de.loc[decomm_coal_idx,'shutdown'] = decomm_coal_de.loc[decomm_coal_idx, 'Geplante Ausserbetriebnahme']

all_decomm_idx = decomm_coal_idx.union(decomm_idx)

### Add exogeneous decommissionings based on unit lifetime or assumption

Steps applied:
* Combine data sets for existing and new-built units with some technology assumptions given in a separate file.
* Unit lifetime is determined based on assumptions. &rarr; _NOTE: Unit lifetimes given here seem to be rather conservative estimates. It is assumed that unit lifetime estimates from the literature can be prolongued by 20 years through retrofits._
* Decommissioning information for new-built plants &rarr; Solely based on unit age except for the one coal plant 'Datteln 4'; no extension of lifetimes is allowed through retrofits.
* Plants which have exceeded their lifetime are to be decomissioned based on unit age: Decommissionings occur in the x years (default: x=10) following a number of years (offset; default=5) after the start year, i.e. the one for which no OPSD data is available anymore. In each year, around 100/x% of the overall capacity to be decommissioned is actually decommissioned, whereby only entire blocks or plants are decommissioned. The order is determined by plant age and electrical efficiency.

In [ ]:
assumptions = pd.read_excel(path_folder_inputs+path_folder_assumptions+filename_assumptions, 
                            sheet_name='assumption table',
                            index_col='Conversion')
assumptions.drop(columns=['min_load_factor'], inplace=True)

conv_de = conv_de.merge(assumptions, left_on='tech_fuel', right_index=True, how='left')
conv_de_new = conv_de_new.merge(assumptions, left_on='tech_fuel', right_index=True, how='left')

# Assumption: power plants without commissioning year were commissioned in 1990
conv_de.loc[conv_de['commissioned_last'].isna(), 'commissioned_last'] = 1990

# start_year is the one following the latest OPSD data state (end of 2018)
# unit lifetimes are assumed to be prolongued by 20 years through retroffiting
start_year = 2019
conv_de['decommissioned_calc'] = conv_de['commissioned_last'] + conv_de['unit_lifetime'] + 20
conv_de.loc[conv_de['shutdown'].isna(), 'shutdown'] = conv_de['decommissioned_calc']

plants_to_decomm = conv_de.loc[~(conv_de['shutdown'].isna()) 
                               & (conv_de['decommissioned_calc'] <= start_year) 
                               & ~(conv_de['fuel'] == 'uranium')]

# New plants: Decommissioned based on unit age; no extension through retrofits
conv_de_new['decommissioned_calc'] = conv_de_new['commissioned_last'] + conv_de_new['unit_lifetime']
conv_de_new.loc[conv_de_new['shutdown'].isna(), 'shutdown'] = conv_de_new['decommissioned_calc']

# Handle plants that already exceeded their calculated lifetimes
plants_to_decomm = plants_to_decomm.sort_values(by=['commissioned_last','efficiency_el'], 
                                                ascending=True)

# x is the number of years over which the decommissionings shall be spread
# offset is the number of years after the start year to start with the decommissionings
x = 10
offset = 5
number_plants = plants_to_decomm.shape[0]
plants_per_year = round(number_plants / x)
counter = 0

for year in range(start_year+offset, start_year+offset+x-1):    
    idx = plants_to_decomm.iloc[counter:counter+plants_per_year].index
    conv_de.loc[idx, 'shutdown'] = year
    counter += plants_per_year

idx = plants_to_decomm.iloc[counter:].index
conv_de.loc[idx, 'shutdown'] = year+offset+x-1

conv_de['shutdown'] = conv_de['shutdown'].astype(int)

Drop the already decommissioned plants as of 2030

In [ ]:
decomm_idx = conv_de[conv_de['shutdown'] < 2030].index
decomm_new_idx = conv_de_new[conv_de_new['shutdown'] < 2030].index
conv_de_2030 = conv_de.drop(decomm_idx)

print("Summary statistics on capacity development for Germany until 2030:")
print(60 * "-")
print(f'Overall capacity to be decommissioned: {round(conv_de.loc[decomm_idx, "capacity"].sum()):,} MW')
print(f'Remaining capacity (excluding new built units): {round(conv_de_2030.capacity.sum()):,} MW')
print(f'Overall capacity of new-built power plants: {round(conv_de_new.capacity.sum()):,} MW')

### Add new-built plants by assumption for preventing loss of load

Please refer to [this section](#Transformers-(addition)) below since data for RES is needed for proper adequacy evaluation.

> _NOTE:_
> * _The overall capacity for 2030 derived from plans for plants to be commissioned is not sufficient to meet the demand._
> * _Hence, the capacity gaps are filled up according to the scheme of the capacity balance (sheet)._

## Assign minimum load profiles
Steps applied:
- Non-CHP and elictricity market-based CHP ('emb') units are assigned a minimum load value of zero, hence, no minimum load is applicable for these units and they can be switched off.
- District heating ('chp') plants and IPPs are assigned a time variant minimum load, depending on heating demand and industry shift profiles, respectively.

**@YW: What about this section?! I see two options:**
* Integrate your clean min loads notebook or
* Leave it separate and remove this section here.

### Industry

In [ ]:
conv_de['min_load_LP'] = 0

In [ ]:
conv_de.loc[conv_de['type'] == 'emb', 'min_load_LP'] = 0

In [ ]:
when2heat = pd.read_csv(path_folder_inputs+path_folder_timeseries+'when2heat.csv', encoding='cp1252', sep=';',
                        usecols=['utc_timestamp', 'DE_heat_demand_total'], parse_dates=['utc_timestamp'],
                        index_col='utc_timestamp')

In [ ]:
def load_entsoe_chp(name):
    load = pd.read_csv('./raw_data_input/timeseries/min_loads/'+name+'.csv', index_col=0, parse_dates=[0])
    df = pd.DataFrame(index=pd.date_range(start='2017-01-01 00:00:00', end='2017-12-31 23:00:00', freq='H', tz='UTC'))
    df = df.join(load)
    df = df.interpolate('linear')
    return df

In [ ]:
district_heat_demand = when2heat['DE_heat_demand_total'].loc['2013-01-01T00:00:00Z':'2013-12-31T23:00:00Z']
district_heat_demand = district_heat_demand/district_heat_demand.max()

min_load_ts = district_heat_demand.to_frame().rename(columns={'DE_heat_demand_total': 'chp'})
# following only applies for oil other fossils etc.
min_load_ts.loc[:, 'chp'] = np.maximum(min_load_ts.loc[:, 'chp'] - 0.5, 0)

In [ ]:
# It seemed liked min loads are too high, so I subtract a bit here
min_load_ts['chp_natgas'] = (load_entsoe_chp('BerlinMitte_BNA0073') / conv_de.loc['BNA0073', 'capacity']).values - 0.1
min_load_ts['chp_lignite'] = 0.5*(
    (load_entsoe_chp('Lippendorf_BNA0115') / conv_de.loc['BNA0115', 'capacity']).values +\
    (load_entsoe_chp('Lippendorf_BNA0116') / conv_de.loc['BNA0116', 'capacity']).values) - 0.1
min_load_ts['chp_hardcoal'] = 0.5*(
    (load_entsoe_chp('ReuterWest_BNA0086') / conv_de.loc['BNA0086', 'capacity']).values +\
    (load_entsoe_chp('ReuterWest_BNA0087') / conv_de.loc['BNA0087', 'capacity']).values) - 0.1
# min_load_ts['ipp'] = 0.5*(
#     (load_entsoe_chp('LudwigshafenSued_BNA0615') / conv_de.loc['BNA0615', 'capacity']).values +\
#     (load_entsoe_chp('LudwigshafenMitte_BNA0614b') / conv_de.loc['BNA0614b', 'capacity']).values)*2 # *2 is arbitrary chosen.
min_load_ts['ipp'] = np.where((min_load_ts.index.hour > 6) & (min_load_ts.index.hour < 22), 0.9, 0.7)
min_load_ts[min_load_ts < 0] = 0

In [ ]:
FR = tools.load_entsoe_generation_data('FR')
AT = tools.load_entsoe_generation_data('AT')

In [ ]:
min_load_ts['FR_natgas'] = np.round(
    FR['Fossil Gas  - Actual Aggregated [MW]'] / FR['Fossil Gas  - Actual Aggregated [MW]'].max(), 4)
min_load_ts['AT_natgas'] = np.round(
    AT['Fossil Gas  - Actual Aggregated [MW]'] / AT['Fossil Gas  - Actual Aggregated [MW]'].max(), 4)

In [ ]:
min_load_ts.index= pd.date_range(start='2017-01-01 00:00:00', end='2017-12-31 23:00:00', freq='H')

min_load_ts = min_load_ts.round(4)
min_load_ts.to_csv(path_folder_outputs+filename_transformers_minload_ts)
min_load_ts.to_excel(writer, sheet_name='min_load_ts')

In [ ]:
#_ = district_heat_demand.plot()
#plt.show()

Basically do the same as above for new built power plants as well

In [ ]:
conv_de_2030['min_load_LP'] = 0
conv_de_new['min_load_LP'] = 0

## Assign gradients and minimum loads

Steps applied:
- Drop columns not needed anymore from data set and do some column renaming
- Add estimate for gradients from assumption table and convert values (from %/min to MW/hour)
- Scale maximum load values for Germany based on historic market data
- Assign minimum load values per technology for German plants

In [ ]:
cols_to_keep = ['country', 'fuel', 'tech_fuel', 'capacity', 
                'efficiency_el', 'type', 'min_load_LP']
cols_to_keep.extend(assumptions.columns)

conv_de.drop(columns=conv_de.columns[~conv_de.columns.isin(cols_to_keep)], 
             inplace=True)

# Assign gradients for Europe separately
conv_eu = conv_eu.merge(assumptions, left_on='tech_fuel', right_index=True, how='left')

conv_eu['grad_pos'] = np.minimum(1, 60 * conv_eu['load_grad_relative'])
conv_eu['grad_neg'] = np.minimum(1, 60 * conv_eu['load_grad_relative'])

conv_eu.drop(columns=['tech_fuel', 'load_grad_relative'], inplace=True)

From exploration of German Entsoe electricity generation data:
- min loads & max feedin (MW) (in total):
    - lignite: [0.249181, 18316.25]
    - hardcoal: [0.03616, 18447.75]
    - nuclear: [0.329853, 10334.25]
    - natural gas: [0.100636, 8684.75]
    - pumped storage, pump: [0, 4475.25]
    - pumped stroage, turbine: [0, 8680.75]

<span style="color:red">**COMMENT BY YW: Important: This needs a rework in case we ensure not minimal loads but fixed ones.**</span>

In [ ]:
# save original aggregated capacites to estimate fixed costs later on
aggregated_capacities_de = conv_de.groupby('fuel').agg({'capacity':sum})

max_load_dict = {'lignite': 18316.25, 
                 'hardcoal': 18447.75, 
                 'uranium': 10334.25, 
                 'natgas': 8684.75+500}

for fuel in max_load_dict.keys():
     conv_de.loc[(conv_de['country'] == 'DE') 
                 & (conv_de['fuel'] == fuel), 'capacity'] *= (
        max_load_dict[fuel] / conv_de.loc[(conv_de['country'] == 'DE') 
                                          & (conv_de['fuel'] == fuel), 'capacity'].sum())

min_load_dict = {'lignite': 0.4, 
                 'hardcoal': 0.25, 
                 'uranium': 0.5, 
                 'natgas': 0.2}

# Convert gradients from %/min to MW/hour (relative values)
conv_de = tools.assign_gradients_and_min_loads(conv_de, min_load_dict)

conv_de_2030.drop(columns=conv_de_2030.columns[~conv_de_2030.columns.isin(cols_to_keep)], 
                 inplace=True)

conv_de_new.drop(columns=conv_de_new.columns[~conv_de_new.columns.isin(cols_to_keep)], 
                 inplace=True)

conv_de_2030 = tools.assign_gradients_and_min_loads(conv_de_2030, min_load_dict) 
conv_de_new = tools.assign_gradients_and_min_loads(conv_de_new, min_load_dict) 

## Prepare conventional power plant data for usage in oemof.solph

After finalizing the conventional power plants data set, this section serves to prepare the data for the usage in the oemof.solph terminology used by _POMMES_. Therefore, European power plant data is aggregated and identifiers as well as electricity buses are created.

### Aggregate European data and create a common data basis (status quo)

Steps applied:
- Aggregate European power plant data by country and fuel
- Derive maximum, minimum and gradient values from historical ENTSO-E data. Maximum gradients are obtained from the 99% percentile of differences in power output between consecutive hours
- Create a common data basis for Germany and Europe by concatenating the detailed German and the aggregated European power plant data

**@YW: Summing up both capacity and gradients seems like duplicated here ...**

In [ ]:
wm = lambda x: np.average(x, weights=conv_eu.loc[x.index, 'capacity'])
agg_dict = dict([(key, (wm)) for key in conv_eu.columns if key not in ['fuel', 'country']])
for key in ['capacity', 'grad_pos', 'grad_neg']:
    agg_dict[key] = 'sum'
agg_dict['type'] = 'first'

conv_eu_agg = conv_eu.groupby(['country', 'fuel'], as_index=False).aggregate(agg_dict)

conv_eu_agg['min_load_factor'] = 0

In [ ]:
# get ENTSOE max, min and max gradient values for conventionals and adjust accordingly
path = '../raw_data_input/hydro/inputs/'
countries = ['AT', 'CH', 'FR', 'NO1', 'NO2', 'NO3', 'NO4', 'NO5', 'NL', 'CZ', 'DK1', 'DK2', 'PL']
generation = {country: tools.load_entsoe_generation_data(country) for country in countries}

fuels = {
    'lignite': 'Fossil Brown coal/Lignite  - Actual Aggregated [MW]',
    'natgas': 'Fossil Gas  - Actual Aggregated [MW]',
    'hardcoal': 'Fossil Hard coal  - Actual Aggregated [MW]',
    'oil': 'Fossil Oil  - Actual Aggregated [MW]',
    'uranium': 'Nuclear  - Actual Aggregated [MW]'
        }
data = []
for c in countries:
    for fuel in fuels.keys():
        try:
            data.append(
                [c,
                 fuel,
                 generation[c][fuels[fuel]].min(),
                 generation[c][fuels[fuel]].max(),
                 generation[c][fuels[fuel]].diff().quantile(0.99)])
        except (KeyError, TypeError): 
            pass        
        
conv_data = pd.DataFrame(columns=['country', 'fuel', 'minimum', 'maximum', 'gradient_max'], data=data)
conv_data['gradient_max'] /= conv_data['maximum']
conv_data['minimum'] /= conv_data['maximum']
# oil has zero maximum infeed
conv_data.drop(index=conv_data[(conv_data['country'] == 'AT')
                               & (conv_data['fuel'] == 'oil')].index, inplace=True)

# overwrite data
for i in conv_data.index:
    c, f = conv_data.loc[i, ['country', 'fuel']]
    conv_eu_agg.loc[(conv_eu_agg['country']==c) & (conv_eu_agg['fuel']==f),
                ['capacity', 'min_load_factor', 'grad_pos']] = (
        conv_data.loc[i, ['maximum', 'minimum', 'gradient_max']].values)
conv_eu_agg['grad_neg'] = conv_eu_agg['grad_pos']

conv_de.loc[conv_de['fuel'] == 'uranium', ['grad_pos', 'grad_neg']] = 0.1729
conv_de.loc[conv_de['fuel'] == 'lignite', ['grad_pos', 'grad_neg']] = 0.2543

In [ ]:
conv = pd.concat([conv_de, conv_eu_agg], sort=False)
conv = conv.reset_index().rename(columns={'index': 'identifier'})
conv.loc[conv['country'] != 'DE', 'identifier'] = np.nan

### Create a European data set for the future

Steps applied:
* Take the capacity information obtained from ENTSOE's TYNDP
* Add the remaining data from the (aggregated) status quo data set
* Handle the differences between the TYNDP and the status quo power plant data:
    * Drop data for plants not occuring in the TYNDP data set (by performing a left merge) &rarr; Capacities not occuring anymore (mostly hardcoal) are implicitly assumed to be shutdown
    * Assign mean values by fuels for the missing values (those contained in the TYNDP data set but not in the status quo data set)
* Add missing information and fill data gaps by mean values per fuel
* Include some assumption on (slight) efficiency improvement due to decommissionings / new commissionings over time (efficiency is assumed to be 5% (not percent points) higher for all fuels)
* Combine all data sets for 2030

In [ ]:
# Set index only for matching purposes
conv_eu_2030.set_index(['country', 'fuel'], inplace=True)
conv_eu_agg.set_index(['country', 'fuel'], inplace=True)

cols_to_concat = [col for col in conv_eu_agg.columns
                  if col not in conv_eu_2030.columns]

conv_eu_2030 = pd.merge(conv_eu_2030, conv_eu_agg[cols_to_concat], left_index=True, right_index=True, how='left')
conv_eu_2030.reset_index(drop=False, inplace=True)

eu_fuels = conv_eu_2030['fuel'].unique()

for fuel in eu_fuels:
    conv_eu_2030.loc[conv_eu_2030['fuel'] == fuel, 
                     :] = conv_eu_2030.loc[conv_eu_2030['fuel'] == fuel, 
                                           :].fillna(conv_eu_2030.loc[conv_eu_2030['fuel'] == fuel, :].mean())
    
conv_eu_2030['type'] = 'emb'
conv_eu_2030.fillna(conv_eu_2030.mean(), inplace=True)
conv_eu_2030['efficiency_el'] = 1.05 * conv_eu_2030['efficiency_el']

In [ ]:
conv_de_2030.loc[conv_de['fuel'] == 'lignite', ['grad_pos', 'grad_neg']] = 0.2543
conv_2030 = pd.concat([conv_de_2030, conv_de_new, conv_eu_2030])
conv_2030 = conv_2030.reset_index().rename(columns={'index': 'identifier'})
conv_2030.loc[conv_2030['country'] != 'DE', 'identifier'] = np.nan

### Introduce a power plant identifier and include connection to oemof.solph elements
Steps applied:
- Introduce an identifier consisting of country, the string 'transformer', fuel type and BNetzA-ID (the latter for Germany only)
- Add a connection to the respective country fuel bus
- Add a connection to the respective country electricity bus
- Write the resulting data set to a csv and and Excel file

> _Note: See the [documentation of oemof.solph](https://oemof.readthedocs.io/en/latest/index.html) for further information on buses which are basically some kind of (balanced) bus bars to connect the elements of an energy system._

In [ ]:
# Existing plants
conv['label'] = np.where(conv['country'] == 'DE', 
                         conv['country'] + '_transformer_' + conv['fuel'] + '_' + conv['identifier'],
                         conv['country'] + '_transformer_' + conv['fuel'])
conv.set_index('label', inplace=True)

#Connection to buses
conv['from'] = conv['country'] + '_bus_' + conv['fuel']
conv['to_el'] = conv['country'] + '_bus_' + 'el'

# Plants as of 2030
conv_2030['label'] = np.where(conv_2030['country'] == 'DE', 
                        conv_2030['country'] + '_transformer_' + conv_2030['fuel'] + '_' + conv_2030['identifier'],
                        conv_2030['country'] + '_transformer_' + conv_2030['fuel'])
conv_2030.set_index('label', inplace=True)

#Connection to buses
conv_2030['from'] = conv_2030['country'] + '_bus_' + conv_2030['fuel']
conv_2030['to_el'] = conv_2030['country'] + '_bus_' + 'el'

### Cluster transformers (optionally)
> _NOTE: This section has been added to create a reduced transformers data for Germany set for quicker, but less precise simulation runs._

Basic steps (only if cluster control variable is True):
* Slice subset for German power plants
* Cluster by electrical efficiency
* Group data based on defined rules
* Assign a new index to clustered data
* Remove detailled data from transformer input sheet and append clustered one instead

In [ ]:
if cluster_transformers_DE:
    conv_de = conv.loc[conv['country'] == "DE"]
    conv_de_clustered = tf_agg.cluster_transformers(
        conv_de, 
        by='efficiency_el', 
        share_clusters=0.1)
    
    grouping_cols = ['cluster']
    mean_cols = ['efficiency_el', 'max_load_factor', 'part_load_losses',
                 'min_uptime', 'min_downtime', 'cold_startup_time', 'warm_startup_time',
                 'hot_startup_time', 'cold_startup_costs', 'warm_startup_costs', 
                 'hot_startup_costs', 'grad_pos', 'grad_neg']
    sum_cols = ['capacity', 'outages']
    cols_considered = grouping_cols + mean_cols + sum_cols

    other_cols = [col for col in conv.columns if col not in cols_considered]

    conv_de_clustered = tf_agg.group_power_plants(
        conv_de_clustered, 
        grouping_cols, 
        mean_cols, 
        sum_cols, 
        other_cols)

    conv_de_clustered['label'] = ('transformer_' 
                                  + conv_de_clustered['fuel'] 
                                  + '_' + conv_de_clustered['type']
                                  + '_cluster_' 
                                  + conv_de_clustered['cluster'].apply(str))
    conv_de_clustered.set_index('label', inplace=True)
    
    conv_clustered = conv.drop(conv_de.index)
    conv_clustered = conv_clustered.append(conv_de_clustered[conv.columns])
    conv_clustered['identifier'] = np.nan
    
    conv_clustered = conv_clustered.round(4)
    conv_clustered.to_csv(path_folder_outputs+'transformers_clustered.csv')
    conv_clustered.to_excel(writer, sheet_name='conv_clustered')

### Write status quo power plants data to csv and Excel

> Note: The 2030 power plants data is written later (see: [this section](#Write-transformer-data-for-2030)) since it includes some adequacy considerations and assumptions about new-built plants to meet the security of supply requirements.

In [ ]:
conv = conv.round(4)
conv.to_csv(path_folder_outputs+filename_transformers)
conv.to_excel(writer, sheet_name='conv')

### Introduce electricity buses
Introduce an electrical bus for every bidding zone modelled. Also see [this section](#Buses) where all buses are written to csv and Excel.

In [ ]:
countries_modeled = ['DE', 'AT', 'BE', 'CH', 'CZ', 'DK1', 'DK2', 'FR', 'NL', 
                     'NO1', 'NO2', 'NO3', 'NO4', 'NO5', 'PL', 'SE1', 'SE2', 'SE3', 'SE4']

buses = [country+'_bus_el' for country in countries_modeled]
buses_2030 = buses.copy()

# Linking Transformers (Interconnectors)

In this section, **interconnector** data for cross-border power exchange modelling is put together.<br>
In [oemof.solph](https://github.com/oemof/oemof-solph) interconnectors for cross-border exchange can be represented by simple transformers elements with one input and one ouput as well as (transportation) losses.
> Note: A former version of _POMMES_ used the custom link component instead.

## Read in interconnector data

A NTC based approach is used in _POMMES_ which considers time-dependent availability of NTCs by applying technical profiles for cross-border exchanges.

The following data is read in:
- Available resp. total transfer capacities for the interconnectors:
    - Data is obtained from Timona Ghosh's master thesis based on the year 2016.
    - Data for exchanges not considered by Timona Ghosh is added from the following sources:
        - Scandinavian countries: [Nordpool](https://www.nordpoolgroup.com/Market-data1/Dayahead/Capacities1/Capacities/KEY/Norway/?view=table), accessed 2018-02-23
        - Belgium: [Elia](https://www.elia.be/en/electricity-market-and-system/electricity-market-facilitation/capacity-calculation)
        - Other countries: ENTSO-E (Forcasted Transfer Capacities - Day Ahead)
- Technical profiles:
    - Technical profiles are derived from Timona Ghosh's master thesis. The original source are publications of the European TSOs and/or correspondence with TSOs.
    - The technical profiles used are those prior to the introduction of FBMC.
- Interconnector timeseries:
    - The time series data contains the timeseries for the parameters used in the technical profiles.
    - The time series are taken from Timona Gosh's master thesis and given for the year 2016. The original source is ENTSO-E resp. TSO transparency information.

> _Note: Technical profiles determine the available NTC capacity which is dependent on wind power generation and/or load. The approach was used prior to flow-based market coupling (FBMC) and is still applied for the exchange with bidding zones which did not yet introduce FBMC. Since the domains for NTC and FBMC are quite similar, NTC still remains a good proxy for cross-border exchange._

In [ ]:
interconn_TTC = pd.read_excel(path_folder_inputs+path_folder_interconnectors+filename_interconnectors_eu,
                              sheet_name = 'interconnector_TTC')

interconn_techprofiles = pd.read_excel(path_folder_inputs+path_folder_interconnectors+filename_interconnectors_eu,
                                       sheet_name = 'technical_profiles', index_col = 'labels')

interconn_ts = pd.read_excel(path_folder_inputs+path_folder_interconnectors+filename_interconnectors_timeseries,
                             parse_dates=True, index_col=0)

## Create data for linking transformer elements

Provide data for transformer elements which model transshipment lines between the country electricity buses.

Steps applied:
-  Create labelling information:
    - name of the linking transformer
    - country bus, the line starts from (exporting country)
    - country bus, the line leads to (importing country)
- Apply a conversion factor of 0.95 in order to prevent excessive exchanges and model losses
- Assign type of link:
    - 'AC': applied for all connections for which technical profiles exist in order to depict time-dependent NTC values
    - 'DC': applied for all other connections for which a fixed value is applied
- Write the resulting data set into a csv and an Excel file

> Example for naming convention used: DE &rarr; AT; name: DE_link_AT

In [ ]:
interconn_TTC['link'] = interconn_TTC['from'] + '_link_' + interconn_TTC['to'] 
interconn_TTC['from'] = interconn_TTC['from'] + '_bus_el'
interconn_TTC['to'] = interconn_TTC['to'] + '_bus_el'

interconn_TTC.drop(columns = 'action', inplace = True)
interconn_TTC['conversion_factor'] = 0.95
interconn_TTC.loc[interconn_TTC.index.isin(interconn_techprofiles.index), 'type'] = 'AC'
interconn_TTC.loc[interconn_TTC['type'] != 'AC', 'type'] = 'DC'

interconn_TTC.set_index('link', inplace = True)

# Ensure limitless capacity to display a single market zone
interconn_TTC.loc['DE_link_AT', [2016, 2017, 2018]] = 100000
interconn_TTC.loc['AT_link_DE', [2016, 2017, 2018]] = 100000

interconn_TTC.to_csv(path_folder_outputs+filename_linking_transformers)
interconn_TTC.to_excel(writer, sheet_name='interconn_TTC')

## Calculate time-dependent NTCs for AC interconnectors

As stated above, the NTC values for AC interconnectors are determined based on technical profiles.

Steps applied:
- A DataFrame holding actual time-dependent NTC values is instanciated
- A factor is introduced to account for interconnector expansion. Example:
    - Max. NTC is 5,000 MW at the moment and it is planned to install another 1,000 MW
    - factor 'network_expansion' has to be set to 6,000 / 5,000 = 1.2
- Functions are introduced for determining the actual NTC value based on the technical profiles
- The actual NTC time series values are set based on the technical profiles, i.e.
    - based on German wind infeed for all German borders except for DE / DK1
    - based on wind infeed and demand in TenneTs control area for DE / DK1
- The NTC timeseries values are written to a csv and an Excel file

In [ ]:
NTC_actual = pd.DataFrame(columns=interconn_techprofiles.index.unique(), index=interconn_ts.index)
network_expansion = 1

year = interconn_ts.index.year
for connector in NTC_actual.columns:
    intercon = interconn_techprofiles.loc[connector].reset_index()
    
    if connector != 'DK1_link_DE' and connector != 'DE_link_DK1':
        wind = interconn_ts['Wind_DE'].to_numpy()
        
        actual_capacity = (
            np.array(
                [tools.find_ntc_wind(intercon, wind, yr) 
                 for yr, wind in np.array([year, wind]).T]) /
            (interconn_TTC.at[connector,2016] * network_expansion))
            
    else:
        wind = interconn_ts['Wind_TenneT'].to_numpy()
        dem = interconn_ts['Demand_TenneT'].to_numpy()
        
        actual_capacity = (
            np.array(
                [tools.find_ntc_load_and_wind(intercon, wind, dem, yr) 
                 for yr, wind, dem in np.array([year, wind, dem]).T]) / 
            (interconn_TTC.at[connector,2016] * network_expansion))                 

    NTC_actual[connector] = actual_capacity
    
NTC_actual.to_csv(path_folder_outputs+filename_linking_transformers_ts)  
NTC_actual.tz_localize(None).to_excel(writer, sheet_name='NTC_actual')

# Storages

In this section, **power storages** data is put together.<br>
In [oemof.solph](https://github.com/oemof/oemof-solph) storages can be represented through (generic) so called "GenericStorage" elements which have specific parameters.

The following types of storages are covered in _POMMES_:
- hydro reservoir storage
- pumped hydro storage

## Reservoir storages

For modelling reservoir energy storages, generation data as well as data for the current (weekly) filling level is used.

Steps applied for preparation:
- Determine path where raw data can be found
- Obtain all files stored in that folder (to read in based on subsets)

In [ ]:
path = path_folder_inputs+path_folder_hydro+'inputs/'
files = [f for f in listdir(path) if isfile(join(path, f))]

### Obtain electricity generation from reservoir storages
Steps applied:
- Read in and prepare generation information from ENTSO-E per bidding zone<br>
(see function `hydro.load_hydro_generation_data` for details)
- Join data from all hydro generation data files and create subsets for reservoir resp. run of river (the latter will be treated later on)

In [ ]:
files_generation = [f for f in files if 'entsoe_generation' in f]
files_generation = [f for f in files_generation if search(('AT|CH|FR|NO'), f)]

generation_df = pd.DataFrame(index=pd.date_range(start='2017-01-01 00:00:00',
                                                 end='2017-12-31 23:00:00', 
                                                 freq='H'))

for file in files_generation:
    bidding_zone = file.partition('generation_')[2][0:2]
    
    if bidding_zone == 'NO':
        bidding_zone = file.partition('generation_')[2][0:3]    
              
    generation_df = generation_df.join(
        hydro.load_hydro_generation_data(bidding_zone=bidding_zone, path=path, filename=file), 
        how='left')
    
generation_reservoir = generation_df[generation_df.columns[generation_df.columns.str.contains('Reservoir')]]
generation_ror = generation_df[generation_df.columns[generation_df.columns.str.contains('ROR')]]

reservoir_load_factors = pd.DataFrame(data={'max_load_factor': generation_reservoir.max(),
                                            'min_load_factor': generation_reservoir.min()})

### Derive an hourly filling rate (time series)

Steps applied:
- Read in and prepare filling rate information from ENTSO-E per bidding zone<br>
(see function `hydro.load_hydro_generation_data` for details)
- Join data from all hydro reservoir data files
- Set first value for filling rate of 2018 as last value for 2017
- Transform weekly values for the filling rate to hourly ones
- Determine minimum and maximum filling rates per country / bidding zone
- Determine inflow of reservoir as:
$$inflow = \Delta filling \space rate + losses + \frac{electricity \space generation}{efficiency}$$
- Determine weekly average of inflow and assign this value for each hour of the week
- Normalize the values by dividing through usable storage energy (which is defined as the difference between max. and min. filling rate)
- Currently, there is no generation data for Sweden, therefore, average data from Norway's bidding zones is taken for the inflow for all four Swedish bidding zones
- Rename columns of DataFrame eventually containing the weekly average inflow on an hourly basis

> *Findings:*
> - *Austria has a small storage capacity, but an extremely high production of hydro. However, the total amount should fit.*
> - *Results are quite sensible to the evaporation rate, and turbine efficiency.*

In [ ]:
hydro_storage_loss_rate = 0.0000
efficiency_turbine = 0.85

files_reservoir = [f for f in files if 'entsoe_hydro_reservoir' in f]

reservoir_df = pd.DataFrame(index=np.arange(0,52,1))

for file in files_reservoir:
    bidding_zone = file.partition('reservoir_')[2][0:2]
    
    if bidding_zone in ['NO', 'SE']:
        bidding_zone = file.partition('reservoir_')[2][0:3]
    
    reservoir_df = reservoir_df.join(
        hydro.load_hydro_reservoir_data(bidding_zone=bidding_zone, 
                                        years=['2017', '2018'], 
                                        path=path, filename=file),
        how='left')

end_values = reservoir_df.loc[0, reservoir_df.columns[reservoir_df.columns.str.contains('2018')]].values
reservoir_df.drop(columns=reservoir_df.columns[reservoir_df.columns.str.contains('2018')], inplace=True)
reservoir_df.index = pd.date_range(start='01-01-2017 00:00:00', periods=52, freq='7D')
reservoir_df.loc[pd.Timestamp('2018-01-01 00:00:00', freq='7D')] = end_values

# Resample to hourly values and obtain max and min values
reservoir_df = reservoir_df.resample('H').interpolate('linear')
min_storageable_energy, max_storageable_energy = (reservoir_df.min(), reservoir_df.max())

In [ ]:
cols_SE = reservoir_df.columns[reservoir_df.columns.str.contains('SE')]
cols_nonSE = reservoir_df.columns[~reservoir_df.columns.isin(cols_SE)]

inflow_df = (reservoir_df.loc[:, cols_nonSE].diff()[1:].values 
    + reservoir_df[:-1][cols_nonSE] * hydro_storage_loss_rate 
    + generation_reservoir.values / efficiency_turbine)

# Do interpolation of inflow (weekly averages)
inflow_df_averaged = inflow_df.apply(lambda x: hydro.upsample_inflow(x))

usable_storage = max_storageable_energy - min_storageable_energy
reservoir_init_level = (reservoir_df.iloc[0] - min_storageable_energy) / usable_storage

rel_avg_hourly_inflow = inflow_df_averaged / usable_storage

### Derive correction factors and estimate Sweden
Steps applied:
* Determine aggregated (theoretical) electricty output of estimated inflow
* For all except for SE, calculate difference of theoretical and real output, and divide by total storage to get a correction for the (hourly) natural inflow
* Subtract this difference from the inflow values
* Use the mean values for Norway as an estimate for Sweden

In [ ]:
# calculate aggregated (theoretical) electricty output os estimated inflow
calculated_energy = (rel_avg_hourly_inflow * usable_storage.values).sum() * efficiency_turbine
calculated_energy.drop(index=calculated_energy.loc[calculated_energy.index.str.contains('SE')].index, inplace=True)

correction_factor = ((calculated_energy - generation_reservoir.sum().values) /
    usable_storage.loc[usable_storage.index.str.contains('AT|CH|FR|NO')].values / 8760)

rel_avg_hourly_inflow.loc[:, rel_avg_hourly_inflow.columns.str.contains('AT|CH|FR|NO')] -= correction_factor.values

cols_NO = reservoir_df.columns[reservoir_df.columns.str.contains('NO')]
rel_avg_hourly_inflow[cols_SE] = (
    np.repeat(rel_avg_hourly_inflow[cols_NO].mean(axis=1).values.reshape((8760,1)), [4], axis=1))

rel_avg_hourly_inflow.columns = [_[0] + '_source_hydro' for _ in rel_avg_hourly_inflow.columns.str.split('_')]

### Prepare data for usage in oemof.solph
Determine parameters for reservoirs such that they can be used to create the respective oemof.solph storages.

Steps applied:
- Put data for usable storage energy amount (nominal_storable_energy) and initial storage state into a DataFrame
- Add loss rate and efficiency for turbine based on assumptions
- Create the hydro sources and buses per country / bidding zone needed for reservoir modelling
- Add sink components for modelling hydro spillage
- Group data per bidding zone
- PL, BE, CZ hydro storages are excluded due to missing date for the filling rates
- Add inflow and outflow bus information
- Set storage type to 'reservoir'

In [ ]:
hydro_countries = [_[0] for _ in usable_storage.index.str.split('_')]
hydro_reservoir_data = pd.DataFrame(
    index=[c+'_storage_el_reservoir' for c in hydro_countries],
    columns=['country','nominal_storable_energy', 'initial_storage_level'],
    data=np.array([hydro_countries, usable_storage.values, reservoir_init_level.values]).T)
hydro_reservoir_data['loss_rate'] = hydro_storage_loss_rate
hydro_reservoir_data.loc['CH_storage_el_reservoir', 'loss_rate'] = 0.00001

hydro_reservoir_data['efficiency_turbine'] = efficiency_turbine
hydro_reservoir_data['efficiency_pump'] = 1

In [ ]:
# Create Hydro sources and buses
sources_hydro = pd.DataFrame(columns=['country', 'capacity'],
                             data=np.array([hydro_countries, usable_storage.values]).T)
sources_hydro = tools.nodes_to_oemof(sources_hydro, to='_bus_hydro', component='_source_hydro')

buses.extend(sources_hydro['to'].to_list())
buses_2030.extend(sources_hydro['to'].to_list())

In [ ]:
sinks_reservoir_spillage = pd.DataFrame(data={'country': hydro_countries,
                                              'from': sources_hydro['to'].to_list()})

sinks_reservoir_spillage['label'] = sinks_reservoir_spillage['country'] + '_sink_hydro_excess'
sinks_reservoir_spillage['excess_costs'] = 1
sinks_reservoir_spillage.set_index('label', inplace = True)

In [ ]:
reservoir_eu_agg = reservoir_eu.groupby(by='bidding_zone', as_index=False).agg({'capacity': 'sum'})
reservoir_eu_agg.drop(index=reservoir_eu_agg[reservoir_eu_agg['bidding_zone'].isin(['BE', 'PL', 'CZ'])].index, 
                      inplace=True)
reservoir_eu_agg = reservoir_eu_agg.set_index(reservoir_eu_agg['bidding_zone'] + '_storage_el_reservoir')

reservoir_eu_agg['bus_inflow'] = reservoir_eu_agg['bidding_zone'] + '_bus_hydro'
reservoir_eu_agg['bus_outflow'] = reservoir_eu_agg['bidding_zone'] + '_bus_el'
reservoir_eu_agg['capacity_pump'] = 100000
reservoir_eu_agg.rename(columns={'capacity': 'capacity_turbine', 'bidding_zone': 'country'}, inplace=True)

reservoir_eu_agg['type'] = 'reservoir'

## Pumped hydro storages

The basic pumped hydro storages information is obtained from the power plants lists (see above).

Steps applied:
- Group the European PHES by bidding zones
- Aggregate the German PHES units as well
- Concatenate both data sets and set storage type to 'phes'
- Add information on inflow and outflow bus as well as capacity information
- Nominal storable energy is based on an assumption: It is assumed that the nominal storable energy equals 4times the plant capacity.

In [ ]:
pumpedstorage_eu_agg = pumpedstorage_eu.groupby('bidding_zone', as_index=False).agg({'capacity': 'sum'})
pumpedstorage_eu_agg = pumpedstorage_eu_agg.set_index(pumpedstorage_eu_agg['bidding_zone'] + '_storage_el_PHS')
pumpedstorage_eu_agg.rename(columns={'bidding_zone': 'country'}, inplace=True)

# There is a large deviation between pumped capacity in AT which is fixed here
pumpedstorage_eu_agg.loc['AT_storage_el_PHS', 'capacity'] =(
    AT['Hydro Pumped Storage  - Actual Aggregated [MW]'].max())

phes_de = phes_de.set_index(phes_de['country'] + '_storage_el_PHS_' + phes_de.index)
phes_de.rename(columns={'capacity_net_bnetza': 'capacity'}, inplace=True)
phes_de.drop(index=phes_de[phes_de['capacity'].isna()].index, inplace=True)

Determine phes plants still operational in 2030
* It is assumed that all plants operational in the status quo will be retroffited instead of shutdown.
* New-built plant(s) is / are added for Germany.
* The European data set is taken as it is given.

In [ ]:
pumpedstorage_eu_2030 = pumpedstorage_eu_2030.pivot(index='country', columns='fuel', values='capacity')
pumpedstorage_eu_2030 = pumpedstorage_eu_2030.T.fillna(pumpedstorage_eu_2030.mean(axis=1)).T

pumpedstorage_eu_2030['label'] = pumpedstorage_eu_2030.index.values + '_storage_el_PHS'
pumpedstorage_eu_2030.reset_index(inplace=True)
pumpedstorage_eu_2030.set_index('label', inplace=True)

pumpedstorage_eu_2030.rename(columns={'PHES_capacity': 'capacity',
                                      'PHES_capacity_pump': 'capacity_pump',
                                      'PHES_capacity_turbine': 'capacity_turbine'}, inplace=True)

In [ ]:
phes_de_2030 = phes_de.copy()
phes_de_2030.rename(columns={'name_bnetza': 'name',
                             'eic_code_plant': 'eic_code'}, inplace=True)
phes_de_2030 = phes_de_2030.append(phes_de_new)

Aggregation for German pumped storages

In [ ]:
# Aggregation for German pumped storages
phes_de_agg = phes_de.groupby('country', as_index=False).agg({'capacity': 'sum'}).rename(
    index={0: 'DE_storage_el_PHS'})

phes = pd.concat([phes_de_agg, pumpedstorage_eu_agg])
phes['type'] = 'phes'

phes['bus_inflow'] = phes['country'] + '_bus_el'
phes['bus_outflow'] = phes['country'] + '_bus_el'
phes['capacity_pump'] = phes['capacity']
phes['capacity_turbine'] = phes['capacity']
phes['nominal_storable_energy'] = 4 * phes['capacity_turbine']
# Values obtained from ENTSOE
phes.loc[phes['country'] == 'DE', ['capacity_pump', 'capacity_turbine']] = [8680.75, 4475.25]
phes.loc[phes['country'] == 'DE', 'nominal_storable_energy'] = 25116

phes.drop(columns='capacity', inplace=True)

Plant status as of 2030

In [ ]:
# Aggregation for German pumped storages
phes_de_2030_agg = phes_de_2030.groupby('country', as_index=False).agg({'capacity': 'sum'}).rename(
    index={0: 'DE_storage_el_PHS'})

phes_2030 = pd.concat([phes_de_2030_agg, pumpedstorage_eu_2030])
phes_2030['type'] = 'phes'

phes_2030['bus_inflow'] = phes_2030['country'] + '_bus_el'
phes_2030['bus_outflow'] = phes_2030['country'] + '_bus_el'
phes_2030['nominal_storable_energy'] = 4 * phes_2030['capacity']
phes_2030.loc[phes_2030['country'] == 'DE', ['capacity_pump', 'capacity_turbine']] = [8680.75, 4475.25]
phes_2030.loc[phes_2030['country'] == 'DE', 'nominal_storable_energy'] = 25116

phes_2030.drop(columns='capacity', inplace=True)

## Add assumptions and store storages data

Steps applied:
- Put together the data for pumped hydro and reservoir storages
- Add missing parameterization for pumped hydro energy storage based on assumptions (i.e. efficiencies, loss rate, ...)
- Save the results in a csv and an Excel file

In [ ]:
storages_el = pd.concat([phes, reservoir_eu_agg], axis=0, sort=False)

assumptions_phes = pd.read_excel(path_folder_inputs+path_folder_assumptions+filename_assumptions,
                                 sheet_name='pumped_storage', index_col=0)

# Set missing parameters for phes based on assumptions
for column in assumptions_phes.index:
    storages_el[column] = assumptions_phes.at[column, 'value']

# Set back values for reservoir storages which have been previously overwritten (again)
storages_el.loc[hydro_reservoir_data.index, hydro_reservoir_data.columns] = hydro_reservoir_data.values
# Restrict maximum and minimum load factors according to historical generation data
reservoir_load_factors.index = [_[0]+'_storage_el_reservoir' for _ in reservoir_load_factors.index.str.split('_')]
storages_el = storages_el.join(reservoir_load_factors, how='left')
storages_el['max_load_factor'] /= storages_el['capacity_turbine']
storages_el['min_load_factor'] /= storages_el['capacity_turbine']
storages_el['max_load_factor'] = storages_el['max_load_factor'].fillna(1)
storages_el['min_load_factor'] = storages_el['min_load_factor'].fillna(0)
storages_el[['max_load_factor', 'min_load_factor']] = storages_el[['max_load_factor', 'min_load_factor']].round(2)

storages_el.to_csv(path_folder_outputs + filename_storages_el)   
storages_el.to_excel(writer, sheet_name='storages_el')

Create data set for 2030:
* In the first place, capacity for Germany is assumed to equal the one in the status quo (for phes and reservoir storages).
* Data for phes for 2030 is given, but capacity values derived from the data set seems unrealistically high and therefore are exogeneously adjusted.

In [ ]:
storages_el_2030 = pd.concat([phes_2030, reservoir_eu_agg], axis=0, sort=False)

# Set missing parameters for phes based on assumptions
for column in assumptions_phes.index:
    storages_el_2030[column] = assumptions_phes.at[column, 'value']

# Set back values for reservoir storages which have been previously overwritten (again) 
storages_el_2030.loc[hydro_reservoir_data.index, hydro_reservoir_data.columns] = hydro_reservoir_data.values

# Restrict maximum and minimum load factors according to historical generation data
reservoir_load_factors.index = [_[0]+'_storage_el_reservoir' for _ in reservoir_load_factors.index.str.split('_')]
storages_el_2030 = storages_el_2030.join(reservoir_load_factors, how='left')
storages_el_2030['max_load_factor'] /= storages_el_2030['capacity_turbine']
storages_el_2030['min_load_factor'] /= storages_el_2030['capacity_turbine']
storages_el_2030['max_load_factor'] = storages_el_2030['max_load_factor'].fillna(1)
storages_el_2030['min_load_factor'] = storages_el_2030['min_load_factor'].fillna(0)
storages_el_2030[['max_load_factor', 
                  'min_load_factor']] = storages_el_2030[['max_load_factor', 'min_load_factor']].round(2)

storages_el_2030.to_csv(path_folder_outputs + filename_storages_el_2030)   
storages_el_2030.to_excel(writer, sheet_name='storages_el_2030')

# Sources

In this section, **sources** data is put together.<br>
In [oemof.solph](https://github.com/oemof/oemof-solph) (generic) "sources" components can be used to represent any kind of energy source. In _POMMES_ sources for the commodities (fuels), for energy shortage and for RES are used. Shortage sources are there to account for situations when overall generation is not sufficient to meet demand. In reality this would result in imports from other European countries and/or activation of reserves as well as forced load sheddings ("brownouts") to prevent a blackout. Shortage sources carry high penalty costs and are used for preserving model feasibility.

## Commodity sources

Steps applied:
- Read in emission factors data
- Extract necessary information from conventional power plants data set, i.e. country, fuel and information on commodity bus (commodity source feeds into that bus; transformer receives energy from that bus)
- Add emission factor information and rename
- Store the results to csv and to Excel
- Add commodity bus information on the buses data set
- Set costs for carbon (hard-coded) and write that cost data

Data sources used:
- Emission factors: UBA (2019). Entwicklung der spezifischen Kohlendioxid-Emissionen des deutschen Strommix in den Jahren 1990 - 2018, Dessau-Roßlau, pp. 16, 28
- Natural gas: BAFA (2019). Aufkommen und Export von Erdgas sowie die Entwicklung der Grenzübergangspreise ab 1991

In [ ]:
emission_factors = pd.read_excel(path_folder_inputs+path_folder_assumptions+filename_emf)

sources_commodity = conv.reset_index()[['country', 'fuel', 'from']].drop_duplicates(subset='from')
sources_commodity['label'] = sources_commodity['country'] + '_source_' + sources_commodity['fuel']
sources_commodity = sources_commodity.merge(emission_factors, on='fuel').drop(columns='fuel').set_index('label')
sources_commodity.rename(columns={'from': 'to'}, inplace=True)

sources_commodity.to_csv(path_folder_outputs+filename_sources_commodity)
sources_commodity.to_excel(writer, sheet_name='sources_commodity')

buses.extend(sources_commodity['to'].values)

costs_carbon = pd.DataFrame(index=sources_commodity.index, columns=range(2015,2050), data=5.8)
costs_carbon.to_csv(path_folder_outputs+filename_costs_carbon)

## Emission limits
Historical emissions as well as political target emission levels can be used to impose optional overall emissions limit.

There are different target emission limits possible:
* BAU prognosis: Simple linear regression based on historical emissions
* 80% reduction (KSP, EK): 80% linear reduction taking into account the sector goal from the Klimaschutzplan 2050 (2016) as well as the 2050 goal from the Energiekonzept (2010)
* 95% reduction (KSP, EK): 95% linear reduction taking into account the sector goal from the Klimaschutzplan 2050 (2016) as well as the 2050 goal from the Energiekonzept (2010)
* 100% reduction (KSG): 100% linear reduction taking into account the sector goals from the Klimaschutzgesetz (KSG) which was agreed upon at the end of 2019 and updated in mid 2021.

Goals:
* KSP 2050: 61-62 % reduction in the energy industry using 466 Mt emissions in 1990 as a starting value which leads to 175-183 Mt GHG emissions in 2030. The mean value (179 Mt) is used. (Table 2 on page 33)
* KSG: Anlage 2 zu § 4: 280 Mt in 2020, 257 Mt in 2022, 108 Mt in 2030; carbon neutrality (which practically means 0 emissions in the energy sector) in 2045 (§ 1 KSG). The goal of 88% emissions reduction until 2040 is not introduced since it is assumed that the contribution of the energy economy sector needs to be higher.

Steps applied:
* Perform a linear regression using statsmodels based on historical emissions (1990-2018) and years.
* Apply historical values to all other possible emission pathways.
* Add the target values as data points for the other pathways and interpolate linearly in between.

Data sources:
* Overall historical emissions: BMWi (2020). Energiedaten Gesamtausgabe. As of 23.10.2020, table 10, line 31 (Energiewirtschaft). https://www.bmwi.de/Redaktion/DE/Binaer/Energiedaten/energiedaten-gesamt-xls.xlsx?__blob=publicationFile&v=131, accessed 23.12.2020.
* Bundesregierung (2010): Energiekonzept für eine umweltschonende, zuverlässige und bezahlbare Energieversorgung. 29. September 2010. https://www.bmwi.de/Redaktion/DE/Downloads/E/energiekonzept-2010.pdf?__blob=publicationFile&v=3, accessed 23.12.2020.
* Bundesregierung (2016): Klimaschutzplan 2050. Klimaschutzpolitische Grundsätze und Ziele der Bundesregierung. https://www.bmu.de/fileadmin/Daten_BMU/Download_PDF/Klimaschutz/klimaschutzplan_2050_bf.pdf, accessed 23.12.2020.
* KSG. Bundes-Klimaschutzgesetz vom 12. Dezember 2019 (BGBl. I S. 2513).

> _NOTE: Setting an emissions limit may conflict resp. interrelates with setting carbon prices. Thus, it is recommended to use carbon prices in the dispatch model (if available) and annual emissions limits in the investment model were minimum loads (of power plant fleets) are considered in a simplified manner._

In [ ]:
GHG_emissions = pd.read_csv(path_folder_inputs+path_folder_assumptions+filename_GHG_emissions,
                            index_col=0, sep=";", decimal=",").T
GHG_emissions.drop(index="Basiswert+", inplace=True)
GHG_emissions.index = GHG_emissions.index.str.strip("*").astype(int)
GHG_emissions.columns = GHG_emissions.columns.str.strip()

# Do a linear regression for the BAU prognosis
X = GHG_emissions.index.values
Y = GHG_emissions.Energiewirtschaft.values

X = sm.add_constant(X)

GHG_regression = sm.OLS(Y, X).fit()

prediction_horizon = range(GHG_emissions.index.values[-1] + 1, 2051)
X_pred = sm.add_constant(prediction_horizon)
GHG_prediction = GHG_regression.predict(X_pred)
GHG_pred = pd.DataFrame(index=prediction_horizon, data={"Energiewirtschaft": GHG_prediction})

GHG_df = GHG_emissions.append(GHG_pred)
GHG_df.columns = ['BAU']
GHG_df['80_percent_linear'], GHG_df['95_percent_linear'], GHG_df['100_percent_linear'] = [GHG_df['BAU'].values] * 3
GHG_df.loc[2019:2050, ['80_percent_linear', '95_percent_linear', '100_percent_linear']] = np.nan

# Assign values for other paths
start_val = GHG_df.at[1990, 'BAU']

emission_targets_KSP_80 = pd.DataFrame({2030: np.mean([175, 183]), 
                                        2050: start_val * (1 - 0.8)},
                                      index=["values"]).T
emission_targets_KSP_95 = pd.DataFrame({2030: np.mean([175, 183]), 
                                        2050: start_val * (1 - 0.95)},
                                      index=["values"]).T
emission_targets_KSG_100 = pd.DataFrame({2020: 280,
                                         2022: 257,
                                         2030: 108,
                                         2045: 0,
                                         2050: 0},
                                       index=["values"]).T

GHG_df.loc[GHG_df['80_percent_linear'].isna(), '80_percent_linear'] = emission_targets_KSP_80['values']
GHG_df.loc[GHG_df['95_percent_linear'].isna(), '95_percent_linear'] = emission_targets_KSP_95['values']
GHG_df.loc[GHG_df['100_percent_linear'].isna(), '100_percent_linear'] = emission_targets_KSG_100['values']
GHG_df.interpolate(method='linear', inplace=True)
# convert million tons of CO2 to tons of CO2
GHG_df = GHG_df.mul(1e6)

GHG_df.to_csv(path_folder_outputs+filename_emission_limits)
GHG_df.to_excel(writer, sheet_name='emission_limits')

## Shortage sources
Steps applied:
- Introduce one electricity shortage source per country modelled and one hydro shortage source for each country with hydro generation
- Concat the two data sets and assign high penalty costs
- Store the results to a csv as well as an Excel file

In [ ]:
sources_shortage_el = pd.DataFrame(data={'country': countries_modeled})
sources_shortage_el = tools.nodes_to_oemof(sources_shortage_el, '_bus_el', '_source_el_shortage')

sources_shortage_hydro = sources_hydro.copy()
sources_shortage_hydro.index = [_+'_shortage' for _ in sources_hydro.index]
sources_shortage_hydro.drop(columns='capacity', inplace=True)

sources_shortage = pd.concat([sources_shortage_el, sources_shortage_hydro])
sources_shortage['shortage_costs'] = 160

sources_shortage.to_csv(path_folder_outputs+filename_sources_shortage)
sources_shortage.to_excel(writer, sheet_name='sources_shortage')

## Renewable sources
In this subsection, **renewable energy sources** (RES) data is put together.<br>
For Germany, a detailled RES modelling approach is used. A distinction between fluctuating and non-fluctuating RES is made.

> _NOTE: Germany and other European bidding zones are treated differently:_
> * _For **Germany**, the sources data set contains the **installed capacities** while the timeseries data set contains maximum values smaller than 1 considering simultaneity of RES infeed._
> * _For other **European** bidding zones, the sources data set contains the **maximum infeed capacity** while the timeseries data set contains maximum values equal to 1._

> _NOTE: The POMMES investment model depicts RES in a much less granular fashion since its only aim is to determine optimal capacity portfolios (as well as overall power system costs). Therefore, the approach laid down in the following is not applied in the investment model._

### Fluctuating RES
Fluctuating RES depicted in _POMMES_ are wind onshore, wind offshore and PV. Run of river (ROR) is modelled seperately (see [reservoir storages section](#Reservoir-storages) for timeseries extraction and [extensions for Germany](#Extensions-for-Germany) for data preparation).

#### Extract timeseries data
Steps applied:
- Read in timeseries data from OPSD for Europe
- Slice values for the year 2017 and do some column renaming for obtaining the oemof.solph terminology
- Rename columns for Skandinavian countries for proper bidding zone terminology (e.g. 'NO1' instead of 'NO_1')

In [ ]:
ts_eu = pd.read_csv(path_folder_inputs+path_folder_timeseries+filename_opsd_timeseries,
                    index_col = 'utc_timestamp')

ts_eu = ts_eu['2017-01-01T00:00:00Z':'2017-12-31T23:00:00Z']
ts_eu.index = pd.to_datetime(ts_eu.index)

dict_rename_ts = {'_wind_onshore_generation_actual': '_source_windonshore',
                  '_wind_offshore_generation_actual': '_source_windoffshore',
                  '_solar_generation_actual': '_source_solarPV',
                  '_load_actual_entsoe_transparency': '_sink_el_load'}

for el in dict_rename_ts:
    ts_eu.columns = ts_eu.columns.str.replace(el, dict_rename_ts[el])

dict_names = dict(zip(ts_eu.filter(regex = 'DK|NO|SE').columns.values,
                      ts_eu.filter(regex = 'DK|NO|SE').columns.str.replace(r"[_]", "", n = 1).values))
ts_eu.rename(dict_names, axis = 1, inplace = True)

ts_eu = ts_eu.tz_localize(None)

#### Do manual data corrections
Steps applied:
- PV infeed in France is below zero for some (night) hours which is obviously a sign of a data error / inconsistency in data formatting &rarr; Hence, PV infeed is forced to 0 for the respective hours.
- For some reason there is no windonshore feedin for the Netherlands. For a rough approximation, the TenneT profile (from DE) is used and scaled with the installed capacity obtained from IRENA (2017)

In [ ]:
ts_eu.loc[ts_eu['FR_source_solarPV'] < 0, 'FR_source_solarPV'] = 0

ts_eu['NL_source_windonshore'] = (
    ts_eu['DE_tennet_wind_generation_actual'] / ts_eu['DE_tennet_wind_generation_actual'].max() * 3245)

#### Prepare RES data for usage in oemof.solph
Steps applied:
- Filter the columns containing RES timeseries data
- Concat the data with ROR generation (has been extracted in hydro (reservoir) data preparation above)
- Extract maximum value for RES generation and set this as "capacity" for European RES
> _Note: What is set as RES "capacity" here is actually the maximum feed in value, taking into account simultaneity of RES generation. Hence, a normalized generation time series with a maximum value of 1 can be derived from that. Actually installed RES generation capacity is higher, though. For Germany, a different approach is chosen and actually installed RES generation capacity is used for time series "normalization". Hence, for Germany, the maximum value of the "normalized" is smaller than 1._

In [ ]:
cols = [c+ts for c in countries_modeled for ts in dict_rename_ts.values()]
ts_eu = ts_eu[ts_eu.columns[ts_eu.columns.isin(cols)]]

ts_eu = pd.concat([ts_eu, generation_ror], axis=1, sort=False)
# NO1 and NO5 have zero windonshore
ts_eu.drop(columns=['NO1_source_windonshore', 'NO5_source_windonshore'], inplace=True)

ts_res = ts_eu.filter(regex = 'wind|PV|ROR')

sources_res = ts_res.max()
sources_res = sources_res.to_frame().rename(columns={0: 'capacity'})

sources_ts_res = ts_res / sources_res['capacity']

### Extensions for Germany
Since Germany is in the focus of the analysis, some extensions are made for the modelling of renewables:
- Non-fluctuating renewables are displayed in more detail
- For flucutating renewables, the bidding strategy is approximated by simulating negative costs

**Approach for modelling marketing of RES**
> In Germany, (fluctuating) RES generation is marketed within the so called "market premium model". Plant operators receive a market premium ($MP$) to compensate for the difference of their individual value applied (more or less equal to their full costs, i.e. LCOE) and the monthly average market revenue at the Day-ahead spot market. So their compensation is
$$Earnings = Spot + MP$$
Hence, it is economically rational to market the generation as long as profit is larger or equal to 0. For flucutating RES, it can be assumed that marginal costs are nearly 0 so that profits are nearly equal to earnings. This leads to:
$$0 \leq Spot + MP$$
and
$$-MP \leq Spot$$
The last inequation means that (rationally acting) plant operators are willing to market their generation as long as spot prices are at least as high as the negative market premium. This rational is modelled in _POMMES_.
Since modelling each RES unit individually would blow up model complexity, power plant clusters for similar market premiums resp. values applicable within the same energy source are created.

Steps applied:
- Read in RES data and group by energy source. Primary data source is TSO data.
- Do some renaming in order to be consistent with the remainder.

Main data sources:
* ÜNB (2018): EEG-Anlagenstammdaten zur Jahresabrechnung 2017, https://www.netztransparenz.de/EEG/Anlagenstammdaten.
* ÜNB (2018): EEG-Bewegungsdaten zur Jahresabrechnung 2017, https://www.netztransparenz.de/EEG/Jahresabrechnungen.

The detailled approach for RES modeling is described in the master's thesis of Yannick Werner (YW):

Werner, Yannick (2020): Modellierung der Auswirkungen einer CO2-Bepreisung auf die EEG-Umlage, master's thesis at the chair of energy and resources economics at TU Berlin.

In [ ]:
usecols = ['eeg_id', 'capacity', 'energy_source', 'commissioning_date', 'support_scheme', 'value_applied']
eeg_pps_de = pd.read_csv(path_folder_inputs+path_folder_renewables+filename_eeg_powerplants,
                         usecols=usecols)

eeg_pps_de_agg = eeg_pps_de.groupby('energy_source').agg({'capacity': 'sum'}) / 1000 # kW to MW

In [ ]:
dict_eeg_pps_names = {'Biomasse': 'biomassEEG',
                      'Deponiegas': 'landfillgas',
                      'Geothermie': 'geothermal',
                      'Grubengas': 'minegas',
                      'Klärgas': 'larga',
                      'Solar': 'solarPV',
                      'Wasser': 'ROR',
                      'Wind_Offshore': 'windoffshore',
                      'Wind_Onshore': 'windonshore'}

eeg_pps_de_agg.rename(index=dict_eeg_pps_names, inplace=True)

#### Run-Of-River
Steps applied and data sources used:
- For Germany, ROR data is extracted from OPSD data
- ROR under the EEG is obtained from the Anlagenstammdaten and added
- The capacity of individual plants is summed up in order to create one representative source
- Read in ROR generation timeseries data, drop NA values and resample from quarter-hourly to hourly frequency
- Normalize the ROR generation profile by dividing through overall installed capacity
- Data source for European ROR plants is (conventional) power plant matching data (see [this section](#Prepare-European-conventional-PP-Raw-Data))

In [ ]:
ror_de_not_eeg = ror_de.loc[(ror_de['status'] == 'operating') 
                            & (ror_de['eeg'] == 'no'), ['name_bnetza', 'country', 'capacity_net_bnetza']]
ror_de_not_eeg.rename(columns={'name_bnetza': 'name', 
                               'capacity_net_bnetza': 'capacity'}, inplace=True)

ror_de_agg = ror_de_not_eeg.groupby(by='country').agg({'capacity': 'sum'})

# Add eeg based capacity
ror_de_agg.loc['DE', 'capacity'] += eeg_pps_de_agg.loc['ROR'].values
ror_de_agg.index = ['DE_source_ROR']

ror_ts_de = pd.read_csv(path_folder_inputs+path_folder_timeseries+filename_entsoe_generation_de,
                        usecols=['Hydro Run-of-river and poundage  - Actual Aggregated [MW]'])
ror_ts_de.columns = ['DE_source_ROR']
ror_ts_de.drop(index=ror_ts_de[ror_ts_de['DE_source_ROR'].isna()].index, 
               inplace=True)
ror_ts_de.index = pd.date_range(start='2017-01-01 00:00:00', 
                                end='2017-12-31 23:45:00', 
                                freq='15min')
ror_ts_de = ror_ts_de.resample('H').mean()

# Calculate ROR profile for Germany
ror_ts_de = ror_ts_de / ror_de_agg.loc['DE_source_ROR', 'capacity']

Estimate 2030 plant status:
* It is assumed, that run-of-river plants are renewed 1:1 instead of being commissioned.
* New-built plant(s) is / are added to the data set.

In [ ]:
ror_de_2030 = ror_de.copy()

ror_de_2030.rename(columns={'capacity_net_bnetza': 'capacity',
                            'eic_code_plant': 'eic_code',
                            'name_bnetza': 'name'}, inplace=True)
ror_de_2030 = pd.concat([ror_de_2030, ror_de_new])

ror_de_2030_not_eeg = ror_de_2030.loc[(ror_de_2030['status'] == 'operating') 
                                      & (ror_de_2030['eeg'] == 'no'), ['name', 'country', 'capacity']]
ror_de_2030_agg = ror_de_2030_not_eeg.groupby(by='country').agg({'capacity': 'sum'})
# Add eeg based capacity
ror_de_2030_agg.loc['DE', 'capacity'] += eeg_pps_de_agg.loc['ROR'].values
ror_de_2030_agg.index = ['DE_source_ROR']

#### Other non-fluctuating RES (EEG)
Steps applied and data sources used:
- Read in power plants capacity and monthly capacity factors which are taken from the master thesis of YW
- Determine the number of hours per month for 2017
- "Roll out" the monthly capacity factors over all hours of a month and assign the hourly time series profile
- Transfer the data to an oemof.solph compatible format

In [ ]:
sources_nonfluc_monthly_capacity_factors = pd.read_csv(
    path_folder_inputs+path_folder_renewables+filename_renewables_nonfluctuating, index_col=0)

sources_nonfluc_ts = sources_nonfluc_monthly_capacity_factors
sources_nonfluc_ts.rename(columns={k: v for k, v in dict_eeg_pps_names.items() 
                                   if k in sources_nonfluc_ts.columns}, inplace=True)

energy_sources = ['biomassEEG', 'landfillgas', 'geothermal', 'minegas', 'larga']
sources_nonfluc = eeg_pps_de_agg.loc[energy_sources].reset_index().rename(
    columns={'energy_source': 'technology'})

sources_nonfluc['country'] = 'DE'

sources_nonfluc = tools.nodes_to_oemof(
    sources_nonfluc, to='_bus_el', component='_source_', component_suffix_var='technology')
sources_nonfluc.drop(columns=['technology'], inplace=True)

sources_nonfluc_ts.columns = sources_nonfluc.index
sources_nonfluc_ts = sources_nonfluc_ts.astype(np.float32)

#### EEG transformers with negative costs
German EEG power plants within the market premium model are modeled as transformers to display negative power prices. (see [section above](#Extensions-for-Germany) for a description of the underlying logic)

Steps applied here:
- Cluster RES power plants by market values and prepare them for usage in _POMMES_ (see function `create_ee_transformers` for details)
- Add RES buses needed for the modelling approach
- Set RES operation costs as negative value of average market premium of cluster
- Write RES data to a csv and to and Excel file
- Write the costs to a separate file

In [ ]:
ee_agg = eeg.create_ee_transformers(eeg_pps_de, cluster_no=3)
ee_agg['country'] = 'DE'
ee_agg['to_el'] = 'DE_bus_el'
ee_agg['efficiency_el'] = 1
buses.extend(ee_agg['from'].unique())

# Seperate costs and nodes
costs_operation_RES = -ee_agg['value_applied'].to_frame().rename(columns={'value_applied': 'costs'})

ee_agg.to_csv(path_folder_outputs+filename_transformers_RES)
ee_agg.to_excel(writer, sheet_name='transformers_RES')
costs_operation_RES.to_csv(path_folder_outputs+filename_costs_operation_res)

#### Other adjustments for RES in Germany
- Overwrite installed RES capacities for Germany with the _true_ ones to derive proper time series (overall capacity of market premium _and_ other (feed-in tariff) RES plants)
- Overwrite German RES time series by dividing through the installed RES capacities (leads to a maximum value smaller than one for the "normalized" infeed time series, see [this section above](#Prepare-RES-data-for-usage-in-oemof.solph)); Result is assigned to RES buses, not sources (since it is provided by multiple units)
- Do some column renaming and fill nan values through linear interpolation
- Overwrite data for German RES power plants from the overall RES data set with the accurate data for Germany
- Write the RES data for Germany to csv
- Load market values from netztransparenz.de and do some data pre-preparations (see function `get_market_values` for details) in order to derive market value time series
- Write the market values time series to csv

In [ ]:
# RES sources, buses and timeseries
fluc_res_de = {'DE_source_solarPV': ('oh_solar', 'DE_bus_solarPV'), 
               'DE_source_windoffshore': ('oh_windoffshore', 'DE_bus_windoffshore'), 
               'DE_source_windonshore': ('oh_windonshore', 'DE_bus_windonshore')}

for key, value in fluc_res_de.items():
    ts_res.loc[:, key] = eeg.load_online_extrapolation(key, value[0])

sources_res.loc[fluc_res_de, 'capacity'] = (
    eeg_pps_de_agg.loc[[key.split("_")[2] 
                        for key in fluc_res_de.keys()], 'capacity'].values)

sources_ts_res[list(fluc_res_de.keys())] = (
    ts_res[list(fluc_res_de.keys())] / sources_res.loc[list(fluc_res_de.keys()), 'capacity'])

sources_ts_res.rename(columns={key: value[1] for key, value in fluc_res_de.items()}, 
                      inplace=True)

# Interpolate NaNs
print('Timeseries that contain NaNs: ', sources_ts_res.columns[sources_ts_res.isna().any()])
sources_ts_res = sources_ts_res.interpolate(method='linear')

sources_res.drop(index=list(fluc_res_de.keys()), inplace=True)

sources_res_fluc_de = pd.DataFrame(index=list(fluc_res_de.keys()), 
                                   data=[value[1] for value in fluc_res_de.values()],      
                                   columns=['to'])
sources_res_fluc_de['country'] = 'DE'
sources_res_fluc_de.index.name = 'label'
sources_res_fluc_de.to_csv(path_folder_outputs+filename_sources_fluc_res)

# Market values
market_values_ts = eeg.get_market_values(netztransparenz=True,
                                         filename=path_folder_inputs+path_folder_renewables+filename_market_values,
                                         year=2017)

market_values_ts.to_csv(path_folder_outputs+filename_costs_market_values)

### Prepare RES data for usage in oemof.solph

Steps applied:
- Put together fluctuating RES data
- Add country information and electricity bus as well as label
- Combine data sets for fluctuating and non-fluctuating RES
- Write results to csv as well as to Excel

In [ ]:
sources_res = pd.concat([sources_res, ror_de_agg])
sources_res['country'] = [c[0] for c in sources_res.index.str.split('_')]
sources_res['to'] = sources_res['country'] + '_bus_el'
sources_res.index.name = 'label'

sources_RES = pd.concat([sources_res, sources_nonfluc, sources_hydro], sort=False)

sources_ts_res = pd.concat([sources_ts_res, ror_ts_de, sources_nonfluc_ts, rel_avg_hourly_inflow], 
                           axis=1, 
                           sort=False)
sources_ts_res = np.round(sources_ts_res, 4).astype(np.float32)

sources_RES.to_csv(path_folder_outputs+filename_sources_RES)
sources_ts_res.to_csv(path_folder_outputs+filename_sources_RES_ts)
sources_RES.to_excel(writer, sheet_name='sources_RES')
sources_ts_res.tz_localize(None).to_excel(writer, sheet_name='sources_RES_ts')

### Renewable capacities and infeed for 2030

Basic steps:
* For German and European RES, capacity expansion factors per RES source are calculated and installed resp. maximum infeed capacities are scaled accordingly.
* European capacities are determined based on IRENA data. For Germany, a study by Prognos et al. is used for capacity projection.
* For Germany, an in-depth analysis of new installations is made in order to derive future values applied. Thereby, the results from the German tender procedures are evaluated which will determine prices for the next few years. In addition to that, cost projections are put together based on literature projections.

#### Determine European capacity development

Steps applied:
* Read in capacity data for the status quo from IRENA
* Determine capacity development from TYNDP for Europe.
* Determine a scaling factor for the capacity values of RES.
    * Use quotient between TYNDP data and capacities from IRENA where applicable.
    * Assign values of 1 for reservoir and ROR plants. (no other information available &rarr; assumed to remain constant)
    * Use quotient between TYNDP and ENTSOE data from above (peak infeed capacities) where no other information is available in IRENA data set.

Data sources:
- Data on capacity of fluctuating RES (except denmark), IRENA (2020): https://www.irena.org/Statistics/Download-Data, data downloaded 09.11.2020; all sources, data for 2017 used.
- Data on fluctuating RES for denmark: ENTSO-E (2019): Generation - Installed Capacity per Production Type, BZN DK1 / DK2, https://transparency.entsoe.eu/dashboard/show, accessed 20.11.2019. Data for recent years seems to underestimate capacities slightly compared to IRENA data.

In [ ]:
sources_RES_cap = pd.read_excel(path_folder_inputs+path_folder_renewables+filename_REcap_eu)
DK_cap = pd.read_excel(path_folder_inputs+path_folder_renewables+filename_REcap_DK)

sources_RES_cap.loc[:,'Technology'] = sources_RES_cap.loc[:,'Technology'].fillna(method='ffill')

sources_RES_cap.rename(columns = {'Country/area': 'country', 'Technology': 'technology'}, 
                       inplace = True)
sources_RES_cap.drop(columns = 'Indicator', inplace = True)
sources_RES_cap['country'].replace({'Europe': 'EU', 'Austria': 'AT', 'Belgium': 'BE', 
                                    'Czechia': 'CZ', 'Denmark': 'DK', 'Finland': 'FI', 
                                    'France': 'FR', 'Germany': 'DE', 'Hungary': 'HU', 
                                    'Luxembourg': 'LU', 'Netherlands': 'NL', 'Norway': 'NO', 
                                    'Poland': 'PL', 'Portugal': 'PO', 'Slovakia': 'SK',
                                    'Spain': 'ES', 'Sweden': 'SE', 'Switzerland': 'CH', 
                                    'UK': 'GB'}, 
                                   inplace = True)

sources_RES_cap['technology'].replace({'Onshore wind energy': 'windonshore', 
                                       'Offshore wind energy': 'windoffshore',
                                       'Solar photovoltaic': 'solarPV'}, 
                                      inplace = True)
sources_RES_cap = sources_RES_cap[(sources_RES_cap['technology'].isin(['windonshore', 'windoffshore', 'solarPV']))
                                  & (sources_RES_cap['country'].isin(countries_modeled))]
sources_RES_cap.columns= sources_RES_cap.columns.astype(str)

DK_cap['technology'].replace({'Wind Onshore': 'windonshore', 
                              'Wind Offshore': 'windoffshore', 
                              'Solar': 'solarPV'}, 
                             inplace = True)
DK_cap = DK_cap[DK_cap['technology'].isin(['windonshore', 'windoffshore', 'solarPV'])]
DK_cap[['2016', '2017', '2018']] = DK_cap[['2016', '2017', '2018']].astype('int32')

sources_RES_cap = pd.concat([sources_RES_cap, DK_cap], sort = True)

sources_RES_cap.drop(columns=['2016','2018'], inplace=True)
sources_RES_cap.drop(index=sources_RES_cap[sources_RES_cap['2017'].isna()].index, inplace=True)
sources_RES_cap.rename(columns={'2017': 'capacity', 
                                'technology': 'fuel'}, inplace=True)

sources_RES_cap['label'] = sources_RES_cap['country'] + '_source_' + sources_RES_cap['fuel']
sources_RES_cap.set_index('label', inplace=True)

In [ ]:
# TYNDP data on RES
renewables_eu_2030.loc[:,'label'] = renewables_eu_2030['country'] + '_source_' + renewables_eu_2030['fuel']
renewables_eu_2030.set_index('label', inplace=True)
renewables_eu_2030.loc[:,'to'] = renewables_eu_2030['country'] + '_bus_el'
renewables_eu_2030.drop(columns='fuel', inplace=True)
renewables_eu_2030 = renewables_eu_2030.sort_values(by='country')

Add missing data for 2030:
* Append hydro reservoir data from the status quo
* Append data for non-fluctuating RES sources for Germany &rarr; Assumed to remain constant, except for biomass for which an update from the Prognos study is introduced.
* Append ROR data from the status quo

In [ ]:
renewables_eu_2030 = pd.concat([renewables_eu_2030, sources_nonfluc, sources_hydro])
# Difference in data sets indices are the ROR plants
ror_idx = set(sources_RES.index) - set(renewables_eu_2030.index)
renewables_eu_2030 = pd.concat([renewables_eu_2030, sources_RES.loc[ror_idx]])

# Set data type to float in order to perform division
renewables_eu_2030.loc[:,'capacity'] = renewables_eu_2030.loc[:,'capacity'].astype(float)
sources_RES_cap.loc[:,'capacity'] = sources_RES_cap.loc[:,'capacity'].astype(float)
sources_RES.loc[:,'capacity'] = sources_RES.loc[:,'capacity'].astype(float)

Determine capacity expansion factors for RES:
* As quotient between capacity for 2030 from TYNDP and status quo (IRENA)
* As constant for hydro and ROR (=1)
* As quotient between capacity for 2030 from TYNDP and status quo RES sources data set from ENTSOE where IRENA data is missing

In [ ]:
RES_cap_exp_factors = renewables_eu_2030['capacity'].div(sources_RES_cap['capacity'])

# Assume hydro and ROR capacities to remain the same
hydro_ix = [ix for ix in RES_cap_exp_factors.index if '_hydro' in ix]
ROR_ix = [ix for ix in RES_cap_exp_factors.index if '_ROR' in ix]

RES_cap_exp_factors.loc[hydro_ix] = 1.0
RES_cap_exp_factors.loc[ROR_ix] = 1.0

nan_ix = RES_cap_exp_factors.loc[RES_cap_exp_factors.isna()].index
to_add_ix = sources_RES.loc[[el for el in nan_ix 
                             if el in sources_RES.index and "DE_" not in el],:].index
RES_cap_exp_factors.loc[to_add_ix] = renewables_eu_2030['capacity'].div(sources_RES['capacity'])

#### Add RES projection for Germany

Add capacity assumption for RES capacity development for Germany:
* Capacity values for 2030 can be either based on
    * a study by Prognos, Öko-Institut and Wuppertal-Institut on behalf of Agora Energiewende, Agora Verkahreswende and Stiftung Klimaneutralitä or
    * EEG 2021 capacity expansion targets.
* Values for water comprise ROR, reservoir as well as phes with natural inflow (same as in BMWi statistics from AGEB). &rarr; ROR part needed here is probably assumed to reamin constant and so is proceeded here.

Steps applied:
* Read in and select data source to be used (Prognos or EEG_2021)
* Filter capacity values for 2030
* Determine capacity expansion factors by dividing the capacities for 2030 through those of the status quo 

Full citations:
* Prognos, Öko-Institut and Wuppertal-Institut (2020): Klimaneutrales Deutschland. In drei Schritten zu null Treibhausgasen bis 2050 über ein Zwischenziel von -65 % im Jahr 2030 als Teil des EU-Green-Deals. Study on behalf of Agora Energiewende, Agora Verkehrswende and Stiftung Klimaneutralität (full version), https://www.agora-energiewende.de/veroeffentlichungen/klimaneutrales-deutschland/, p. 53, accessed 09.11.2020.
* EEG 2021: Gesetz für den Ausbau erneuerbarer Energien (Erneuerbare-Energien-Gesetz - EEG 2021). Erneuerbare-Energien-Gesetz vom 21. Juli 2014 (BGBl. I S. 1066), das zuletzt durch Artikel 1 des Gesetzes vom21. Dezember 2020 (BGBl. I S. 3138) geändert worden ist.

In [ ]:
res_DE_EEG_2021 = pd.read_excel(path_folder_inputs+path_folder_renewables+filename_RES_DE_EEG_2021,
                                sheet_name='capacities')

res_DE_Prognos = pd.read_excel(path_folder_inputs+path_folder_renewables+filename_RES_DE_Prognos,
                               sheet_name='capacities')

res_capacity_projections = {"Prognos": res_DE_Prognos,
                            "EEG_2021": res_DE_EEG_2021}

if res_capacity_projection not in res_capacity_projections.keys():
    raise ValueError("`res_capacity_projection` has to be either 'Prognos' or 'EEG_2021'.")

res_de_projection = res_capacity_projections[res_capacity_projection]

In [ ]:
res_de_projection['country'] = 'DE'
res_de_projection['to'] = res_de_projection['country'] + '_bus_el'
res_de_projection['label'] = res_de_projection['country'] + '_source_' + res_de_projection['fuel']

res_de_projection_2030 = res_de_projection[res_de_projection['year'] == 2030].copy()
res_de_projection_2030['capacity'] = res_de_projection_2030.loc[:,'capacity [GW]'] * 1000
res_de_projection_2030.drop(columns=['capacity [GW]', 'year'], inplace=True)
res_de_projection_2030.set_index('fuel', inplace=True)

# Replace water capacity by ROR capacity which in turn is assumed to remain constant
res_de_projection_2030.rename(index={'water': 'ROR'}, inplace=True)
res_de_projection_2030.loc['ROR', 
                           ['country', 'to', 
                            'label', 'capacity']] = ['DE', 'DE_bus_el', 'DE_source_ROR',
                                                     eeg_pps_de_agg.at['ROR', 'capacity']]

# Determine capacity development factors for Germany
RES_cap_exp_factors_DE = res_de_projection_2030['capacity'].div(eeg_pps_de_agg['capacity']).fillna(1)
#RES_cap_exp_factors_DE.loc["biomass"] = res_de_projection_2030.at['biomassEEG', 'capacity'] / (
#    eeg_pps_de_agg.at['biomassEEG', 'capacity'])
#RES_cap_exp_factors_DE.drop(index="biomassEEG", inplace=True)
RES_cap_exp_factors_DE.rename(index={ix: 'DE_source_'+ix 
                                     for ix in RES_cap_exp_factors_DE.index}, inplace=True)

RES_cap_exp_factors_fluc_DE = RES_cap_exp_factors_DE.loc[list(fluc_res_de.keys())]
RES_cap_exp_factors_nonfluc_DE = RES_cap_exp_factors_DE.drop(RES_cap_exp_factors_fluc_DE.index)

#### Scale (up) capacities for renewable infeed

Renewable infeed for 2030 is determined by a simple scaling the renewable capacities by the capacity expansion factors (capacity in 2030 / capacity in 2017) determined above:
* The capacities of the RES sources are multiplied by the capacity expansion factors.<br>
_NOTE: While they represent installed capacities for Germany, for other European countries they represent the peak infeed capacity._
* The infeed timeseries themselves are not modified, but taken as they are.
* For RES not contained in the status quo, timeseries of neighbouring countries are assigned as a proxy.

In [ ]:
# Update capacity factors of non-fluctuating RES sources for Germany
RES_cap_exp_factors.loc["DE_source_biomassEEG"] = RES_cap_exp_factors_nonfluc_DE.loc["DE_source_biomassEEG"]
RES_cap_exp_factors.loc[RES_cap_exp_factors_nonfluc_DE.index] = RES_cap_exp_factors_nonfluc_DE

sources_RES_2030 = sources_RES.copy()
sources_RES['capacity'] = sources_RES['capacity'].astype(float)
sources_RES_2030['capacity'] = sources_RES['capacity'].mul(RES_cap_exp_factors)
sources_ts_res_2030 = sources_ts_res.copy()

There are some bidding zones for which no capacity information for certain RES for the status quo is available.
Thus, no timeseries information exists.

As a first proxy, these market zones will simply be assigned timeseries from neighbouring countries:
* FR offshore &rarr; DE offshore
* NO2-5 & SE1-4 solarPV &rarr; DK solarPV
* PL solarPV and offshore &rarr; DE solarPV and offshore
* SE4 offshore &rarr; DK offshore

In [ ]:
missing_res_ts_dict = {
    'FR_source_windoffshore': 'DE_bus_windoffshore',
    'NO2_source_solarPV': 'DK1_source_solarPV',
    'NO3_source_solarPV': 'DK1_source_solarPV',
    'NO4_source_solarPV': 'DK1_source_solarPV',
    'NO5_source_solarPV': 'DK1_source_solarPV',
    'NO5_source_windonshore': 'DK1_source_windonshore',
    'PL_source_solarPV': 'DE_bus_solarPV',
    'PL_source_windoffshore': 'DE_bus_windoffshore',
    'SE1_source_solarPV': 'DK2_source_solarPV',
    'SE2_source_solarPV': 'DK2_source_solarPV',
    'SE3_source_solarPV': 'DK2_source_solarPV',
    'SE4_source_solarPV': 'DK2_source_solarPV',
    'SE4_source_windoffshore': 'DK2_source_windoffshore'}

# Assign missing timeseries and capacity values
German_vals = []
for k, v in missing_res_ts_dict.items():
    sources_ts_res_2030[k] = sources_ts_res.loc[:,v]
    sources_RES_2030.loc[k] = renewables_eu_2030.loc[k] 
    if 'DE_' in v:
        German_vals.append(k)

# Rescaling for German values needed (for the sake of consistency)
for el in German_vals:
    sources_RES_2030.at[k, 'capacity'] = sources_RES_2030.at[k, 'capacity'] * sources_ts_res_2030[el].max()
    sources_ts_res_2030.loc[:,el] = sources_ts_res_2030[el].div(sources_ts_res_2030[el].max())

In [ ]:
sources_RES_2030.to_csv(path_folder_outputs+filename_sources_RES_2030)
sources_ts_res_2030.to_csv(path_folder_outputs+filename_sources_RES_ts_2030)
sources_RES_2030.to_excel(writer, sheet_name='sources_RES_2030')
sources_ts_res_2030.tz_localize(None).to_excel(writer, sheet_name='sources_RES_ts_2030')

### Extension for Germany: Determine new-built RES capacities and values applied
General approach:
* Add plants commissioned until 2030 and introduce an assumption for the values applied by year.
* For commissionings until 2022 (2025 for offshore) the current results from the EEG tender procedures (capacities and prices) are used.
* For commissionings from 2023 on (2026 for offshore), an assumption for the capacities to be commissioned is made. Therefore, the target capacity to be build is split equally onto the years from 2023 to 2030.
* Thereby, decommissionings based on unit age have to be taken into account. It is assumed that plants are decommissioned after 20 years of operation.

#### Determine decommissioning years and calculate (gross) capacity expansion needed

Determine amount of RES capacity to be installed for every year:
* Decommissioning of EEG plants will happen from 2021 on after 20 years of operation.
    * For the sake of simplicity, use commissioning year only and only stick to the rules from prior EEG versions (until EEG 2014) which granted payments for 20 years + remainder of the commissioning year. &rarr; Assume 21 years lifetime.
    * Determine the remaining capacities by energy source and store them in a dictionary indexed by years.
* Determine the amount of capacity commissioned until 2022 by already finished RES tenders.
* Determine the amount of (gross) capacity to be added to fill up the gap up to the installed capacities for 2030 given in the study from Prognos et al. (2020) or the EEG 2021.

In [ ]:
eeg_pps_de['commissioning_year'] = eeg_pps_de['commissioning_date'].astype(str).str.slice(start=0, stop=4).astype(int)
eeg_pps_de['decommissioning_year'] = np.where(eeg_pps_de['commissioning_year'] + 21 > 2020, 
                                              eeg_pps_de['commissioning_year'] + 21, 2021)

In [ ]:
# Determine the amount of installed capacity remaining for every year
eeg_pps_de_agg_by_year = {}

to_group = eeg_pps_de[eeg_pps_de['decommissioning_year'] > 2030]
eeg_pps_de_agg_2030 = to_group.groupby('energy_source').agg({'capacity': 'sum'}) / 1000 # kW to MW
eeg_pps_de_agg_2030.rename(index=dict_eeg_pps_names, inplace=True)
    
cap_needed_until_2030 = res_de_projection_2030['capacity'].sub(eeg_pps_de_agg_2030['capacity']).fillna(0)

#### Include data from German RES tenders

* Read in the data for the different energy sources.
* Decrease the amount of capacity needed by
    * capacity to be commissioned from the EEG tender procedures and
    * capacity installed elsewhise (only for solarPV).
* Obtain values applied from the tender data.

_NOTE: For the sake of simplicity, the deviating deadlines for Bürgerenergiegesellschaften are neglected for wind onshore._

RES tenders for wind onshore

In [ ]:
onshore_tenders = pd.read_excel(path_folder_inputs+path_folder_renewables+filename_onshore_tenders,
                                sheet_name='Alle Runden', skiprows=5, nrows=20, index_col=0, 
                                usecols=[0, 8, 10, 11, 12, 20])

dict_tender_cols = {'Zuschlags-menge (kW)': 'capacity_awarded',
                    'Zuschlagswerte (ct/kWh)': 'min',
                    'Unnamed: 11': 'max',
                    'Unnamed: 12': 'weighted_ave',
                    'Frist zur Inbetriebnahme ohne Fördersatzreduktion': 'due_date'}
onshore_tenders.rename(columns=dict_tender_cols, inplace=True)
onshore_tenders = onshore_tenders[~(onshore_tenders.index.isna())]

# Due to the EEG, deadline for commissioning is 24 months for the tenders in 2019
onshore_tenders.loc[onshore_tenders.due_date.isna(), 
                    'due_date'] = (onshore_tenders.loc[onshore_tenders.due_date.isna()].index
                                   + pd.DateOffset(months=24) + pd.tseries.offsets.MonthEnd(1))

# Determine weigthed averages per commissioning year
onshore_tenders['tender_year'] = onshore_tenders.index.year
onshore_tenders['commissioning_year'] = pd.DatetimeIndex(onshore_tenders.due_date).year

wm = lambda x: np.average(x, weights=onshore_tenders.loc[x.index, 'capacity_awarded'])

onshore_tenders.loc[:,['min', 'max', 'weighted_ave']] = (
    onshore_tenders.loc[:,['min', 'max', 'weighted_ave']].astype(float))

windonshore_commissionings = onshore_tenders.groupby('commissioning_year').aggregate({
    'capacity_awarded':'sum',
    'min': wm, 'max': wm, 'weighted_ave': wm})

RES tenders for PV and common RES tender (wind + PV):
* So far, only PV plants have been successful within the common tender procedure.
* Hence, the capacities are added onto the PV bids.

In [ ]:
solarPV_tenders = pd.read_excel(path_folder_inputs+path_folder_renewables+filename_solar_tenders,
                                sheet_name='Alle Runden', skiprows=5, nrows=25, index_col=0, 
                                usecols=[1, 9, 11, 12, 13, 19])

common_tenders = pd.read_excel(path_folder_inputs+path_folder_renewables+filename_common_tenders,
                               sheet_name='Alle Runden', skiprows=5, nrows=6, index_col=0, 
                               usecols=[0, 8, 10, 11, 12, 18])

dict_tender_cols = {'Zuschlagsmenge (MW)': 'capacity_awarded',
                    'Zuschlagswerte (ct/kWh)': 'min',
                    'Unnamed: 12': 'max',
                    'Unnamed: 13': 'weighted_ave',
                    'Frist zum Antrag auf Zahlungsberechtigung2 ohne Fördersatzreduktion': 'due_date'}
solarPV_tenders.rename(columns=dict_tender_cols, inplace=True)

dict_tender_cols = {'Zuschlagsmenge (MW)': 'capacity_awarded',
                    'Zuschlagswerte (ct/kWh)': 'min',
                    'Unnamed: 11': 'max',
                    'Unnamed: 12': 'weighted_ave',
                    'Frist zum Antrag auf Zahlungsberechtigung2 ohne Fördersatzreduktion\n - bei Solar -': 'due_date'}
common_tenders.rename(columns=dict_tender_cols, inplace=True)

solarPV_tenders = solarPV_tenders[~(solarPV_tenders.index.isna())]
solarPV_tenders = solarPV_tenders[~(solarPV_tenders.index.duplicated(keep='first'))]
common_tenders = common_tenders[~(common_tenders.index.isna())]
solarPV_tenders = solarPV_tenders.append(common_tenders)

solarPV_tenders.loc[solarPV_tenders.due_date.isna(), 
                    'due_date'] = (solarPV_tenders.loc[solarPV_tenders.due_date.isna()].index
                                   + pd.DateOffset(months=24) + pd.tseries.offsets.MonthEnd(1))

# Determine weigthed averages per commissioning year
solarPV_tenders['tender_year'] = solarPV_tenders.index.year
solarPV_tenders['commissioning_year'] = pd.DatetimeIndex(solarPV_tenders.due_date).year

wm2 = lambda x: np.average(x, weights=solarPV_tenders.loc[x.index, 'capacity_awarded'])
solarPV_tenders.loc[:,['min', 'max', 'weighted_ave']] = (
    solarPV_tenders.loc[:,['min', 'max', 'weighted_ave']].astype(float))

solarPV_commissionings = solarPV_tenders.groupby('commissioning_year').aggregate({
    'capacity_awarded':'sum',
    'min': wm2, 'max': wm2, 'weighted_ave': wm2})
solarPV_commissionings.drop([2016, 2017], inplace=True)

RES tenders for wind offshore (transitional model):

Data sources:
* BNetzA (2017): BK6-17-001, Ergebnisse der 1. Ausschreibung vom 01.04.2017, https://www.bundesnetzagentur.de/DE/Service-Funktionen/Beschlusskammern/1_GZ/BK6-GZ/2017/BK6-17-001/Ergebnisse_erste_Ausschreibung.pdf?__blob=publicationFile&v=3, Abruf am 17.11.2020.
* BNetzA (2018): BK6-18-001, Ergebnisse der 2. Ausschreibung vom 01.04.2018, https://www.bundesnetzagentur.de/DE/Service-Funktionen/Beschlusskammern/1_GZ/BK6-GZ/2018/BK6-18-001/Ergebnisse_zweite_ausschreibung.pdf?__blob=publicationFile&v=3, Abruf am 17.11.2020.

In [ ]:
windoffshore_commissionings = pd.DataFrame(
    index=['2024', '2025'],
    data={'capacity_awarded': [1490000, 1610000], 
          'min': [0, 0],
          'max': [6, 9.83],
          'weighted_ave': [0.44, 4.66]})

windoffshore_commissionings.index.name='commissioning_year'

Decrease capacity needed until 2030 by the amount of capacity already (to be) installed from the tender procedures

In [ ]:
cap_needed_until_2030.loc['windonshore'] -= windonshore_commissionings['capacity_awarded'].div(1000).sum()
cap_needed_until_2030.loc['windoffshore'] -= windoffshore_commissionings['capacity_awarded'].div(1000).sum()
cap_needed_until_2030.loc['solarPV'] -= solarPV_commissionings['capacity_awarded'].div(1000).sum()

Add capacity from PV build outside the tendering procedures:
* rooftop PV plants, tenant electricity plants ("Mieterstrom") and smaller open space plants &rarr; The value applied is determined by the gross capacity expansion of those plants for which in turn the corrected expansion values are used.
* The capacity values and remuneration values are read in, whereby the values are adjusted every quarter based on the rolling half year before.
* The capacity values are aggregated per year and subtracted from the capacity exapansion need for solarPV until 2030

Data sources:
* BNetzA (2020): EEG-Registerdaten und Fördersätze, https://www.bundesnetzagentur.de/DE/Sachgebiete/ElektrizitaetundGas/Unternehmen_Institutionen/ErneuerbareEnergien/ZahlenDatenInformationen/EEG_Registerdaten/EEG_Registerdaten_node.html;jsessionid=C9A499BB48D4F32668B73B6769FBBD84, Abruf am 19.11.2020.
* BNetzA (2020): Archiv EEG-Vergütungssätze und Datenmeldungen, https://www.bundesnetzagentur.de/DE/Sachgebiete/ElektrizitaetundGas/Unternehmen_Institutionen/ErneuerbareEnergien/ZahlenDatenInformationen/EEG_Registerdaten/ArchivDatenMeldgn/ArchivDatenMeldgn_node.html, Abruf am 19.11.2020.

In [ ]:
pv_cap_names_dict = {'2018_01': filename_PV_2018_01,
                     '2019_01': filename_PV_2019_01,
                     '2019_02': filename_PV_2019_02,
                     '2020_01': filename_PV_2020_01,
                     '2020_02': filename_PV_2020_02,
                     '2021_01': filename_PV_2021_01}

pv_cap_dict = {}

pv_capacities = pd.DataFrame(columns=['capacity'])
pv_capacities.index.name = 'year'

pv_col_names = {'Leistung (kWp)': 'capacity',
                'Leistung (kWp) 1': 'capacity'}

for k, v in pv_cap_names_dict.items():
    # Extract capacity installations info
    pv_cap_dict[k] = pd.read_excel(path_folder_inputs+path_folder_renewables+v,
                                   skiprows=6, usecols="B:C", nrows=6, index_col=0)

    if not isinstance(pv_cap_dict[k].index, pd.DatetimeIndex):
        pv_cap_dict[k].index = pv_cap_dict[k].index.str.strip('*')
        pv_cap_dict[k]['year'] = pv_cap_dict[k].index.str[-4:]
    else:
        pv_cap_dict[k]['year'] = pv_cap_dict[k].index.year
    
    pv_cap_dict[k].rename(columns=pv_col_names, inplace=True)
    pv_capacities = pv_capacities.append(pv_cap_dict[k].groupby('year').agg('sum'))
    
pv_capacities.reset_index(inplace=True)
pv_capacities['year'] = pv_capacities['year'].astype(str)
pv_capacities_agg = pv_capacities.groupby('year').agg('sum').div(1000)

# Drop values for 2017 (already given); Scale installations for 2020
pv_capacities_agg.drop(index='2017', inplace=True)
pv_capacities_agg.at['2020', 'capacity'] = pv_capacities_agg.at['2020', 'capacity'] * (4/3)

cap_needed_until_2030.loc['solarPV'] -= pv_capacities_agg['capacity'].sum()

Distribute the capacity needed (gross) installations:
* Gross capacity installations have to take place between 2023 and 2030 for wind and solarPV
* For windoffshore, gross installations equal the net installations and take place between 2026 and 2030
* Installations from tender procedures are given for PV &rarr; I.e., remaining rooftop installations needed for PV for 2021 and 2022 are given by the annual capacity expansion need minus the installations from the tender procedures

In [ ]:
annual_cap_expansion = pd.DataFrame(dtype=float, index=['windonshore', 'solarPV', 'windoffshore'],
                                    columns=['capacity', 'start_year'])

annual_cap_expansion.loc['windonshore', 
                         'solarPV', 
                         'windoffshore'] = [(cap_needed_until_2030.loc['windonshore'] / (2030 - 2022), 2023), 
                                            (cap_needed_until_2030.loc['solarPV'] / (2030 - 2020), 2021),
                                            (cap_needed_until_2030.loc['windoffshore'] / (2030 - 2025), 2026)]

pv_capacities_agg.loc['2021'] = (annual_cap_expansion.at['solarPV', 'capacity'] 
                                - solarPV_commissionings.at[2021, 'capacity_awarded'] / 1000)
pv_capacities_agg.loc['2022'] = (annual_cap_expansion.at['solarPV', 'capacity'] 
                                - solarPV_commissionings.at[2022, 'capacity_awarded'] / 1000)
pv_capacities_agg.index = pv_capacities_agg.index.astype(int)

#### Add capacities and values applied from the RES tender scheme
Determine remaining capacities in 2030:
* Filter out plants with a decommissioning year prior to 2030.
* Include DataFrames holding artificial new-built units. Artificial new-built units hereby means a collection of units to be newly installed which are grouped by their values applied due to a given capacity distribution.

In [ ]:
eeg_pps_de_2030 = eeg_pps_de[eeg_pps_de['decommissioning_year'] > 2030]

eeg_pps_de_solarPV_2030 = pd.DataFrame(columns=eeg_pps_de.columns)
eeg_pps_de_windonshore_2030 = pd.DataFrame(columns=eeg_pps_de.columns)
eeg_pps_de_windoffshore_2030 = pd.DataFrame(columns=eeg_pps_de.columns)

In [ ]:
del eeg_pps_de

Apply a function to determine capacity distribution as well as another function to create new-built units:
* The data sets each contain a lower, a middle and an upper value for the value applied.
* A beta distribution function for the capacity shares is introduced and the corresponding equally sliced values applied are attributed to the corresponding capacity shares.
* Add power plants based on capacity distribution to the new-built DataFrame.

In [ ]:
# Create capacity distributions and corresponding values applied
PV_cap_distribution = eeg.create_cap_distribution_from_beta(
    solarPV_commissionings,
    cols=['min', 'weighted_ave', 'max'])
                                              
onshore_cap_distribution = eeg.create_cap_distribution_from_beta(
    windonshore_commissionings,
    cols=['min', 'weighted_ave', 'max'])
                                              
offshore_cap_distribution = eeg.create_cap_distribution_from_beta(
    windoffshore_commissionings,
    cols=['min', 'weighted_ave', 'max'])

# Create new-built units with the corresponding market values
eeg_pps_de_solarPV_2030 = eeg.add_new_built_res(PV_cap_distribution, eeg_pps_de_solarPV_2030, 
                                                solarPV_commissionings, 'Solar', 
                                                capacity_col='capacity_awarded', indexed_by_years=True)

eeg_pps_de_windonshore_2030 = eeg.add_new_built_res(onshore_cap_distribution, eeg_pps_de_windonshore_2030, 
                                                    windonshore_commissionings, 'Wind_Onshore', 
                                                    capacity_col='capacity_awarded', indexed_by_years=True)

eeg_pps_de_windoffshore_2030 = eeg.add_new_built_res(offshore_cap_distribution, eeg_pps_de_windoffshore_2030, 
                                                     windoffshore_commissionings, 'Wind_Offshore', 
                                                     capacity_col='capacity_awarded', indexed_by_years=True)

#### Add exogeneous assumptions for development of RES value applied

The estimates are based on the evaluation put together by Conrad Nicklisch in his bachelor's thesis. Primary data sources used are the following ones:
* sources for all energy sources:
    * Greenpeace (2017): Comparing electricity production costs of renewables to fossil and nuclear power plants in G20 countries.
    * Fraunhofer ISE (2018): Stromgestehungskosten Erneuerbare Energien.
    * Reiner Lemoine Institut gGmbh (2013): Vergleich und Optimierung von zentral und dezentral orientierten Ausbaupfaden zu einer Stromversorgung aus Erneuerbaren Energien in Deutschland.
    * Prognos, EWI, GWS (2014): Entwicklung der Energiemärkte - Energiereferenzprognose.
* additional sources for solarPV (open space):
    * Fraunhofer ISE (2015): Current and Future Cost of Photovoltaics, im Auftrag von Agora Energiewende.
* additional sources for wind onshore and wind offshore:
    * DIW (2013): Current and Prospective Costs of Electricity Generation until 2050.
    * DIW, GWS, DLR, Prognos, ZSW (2015): Beschäftigung durch erneuerbare Energien in Deutschland.
    * DLR, Fraunhofer IWES, IfnE (2012): Langfristszenarien und Strategien für den Ausbau der erneuerbaren Energien in Deutschland bei Berücksichtigung der Entwicklung in Europa und global.
    * Greenpeace International (2015): Energy Revolution - A sustainable world energy outlook 2015.
* additional source for wind offshore:
    * Baum, Sergej et al. (2018): Analysis and Modelling of the Future Electricity Price Development by taking the Levelized Cost of Electricity and large Battery Storages into Account.

In [ ]:
windonshore_cost_estimates = pd.read_excel(path_folder_inputs+path_folder_renewables+filename_RES_cost_projections,
                                           sheet_name="Wind Onshore", skiprows=3).drop(index=[0, 1])

windoffshore_cost_estimates = pd.read_excel(path_folder_inputs+path_folder_renewables+filename_RES_cost_projections,
                                           sheet_name="Wind Offshore", skiprows=3).drop(index=[0, 1])

solarPV_cost_estimates = pd.read_excel(path_folder_inputs+path_folder_renewables+filename_RES_cost_projections,
                                           sheet_name="PV Freifläche", skiprows=3).drop(index=[0, 1])

Determine LCOE estimates:
* Calculate the LCOE estimates based on CAPEX, OPEX, full load hours, wacc and unit lifetimes using a function
* Obtain and filter the LCOE projection data to only cover the years that we are interested in.

In [ ]:
dict_RES_cols = {'Investionsausgaben (Capex) - niedrig': 'capex_low',
                 'Investionsausgaben (Capex) - mittel': 'capex_middle',
                 'Investionsausgaben (Capex) -hoch': 'capex_high',
                 'fixe Betriebs- \n& Wartungskosten (Opex_fixed)': 'opex_percentage',
                 'WACC': 'wacc',
                 'Lebensdauer\nder Anlage': 'unit_lifetime',
                 'Volllast-\nstunden': 'flh_low', 
                 'Volllast-\nstunden.1': 'flh_middle', 
                 'Volllast-\nstunden.2': 'flh_high'}

LCOE_windonshore = eeg.estimate_lcoe(windonshore_cost_estimates, dict_RES_cols)
LCOE_windoffshore = eeg.estimate_lcoe(windoffshore_cost_estimates, dict_RES_cols)
LCOE_solarPV = eeg.estimate_lcoe(solarPV_cost_estimates, dict_RES_cols)

LCOE_windonshore = LCOE_windonshore.loc[
    LCOE_windonshore.index >= annual_cap_expansion.at['windonshore', 'start_year']]
LCOE_windoffshore = LCOE_windoffshore.loc[
    LCOE_windoffshore.index >= annual_cap_expansion.at['windoffshore', 'start_year']]
LCOE_solarPV = LCOE_solarPV.loc[
    LCOE_solarPV.index >= annual_cap_expansion.at['solarPV', 'start_year']]

Use latest ISE LCOE data to update the cost data from the sources listed above which are partly outdated:

* Fraunhofer ISE (2020): Supplementary data to the study "Wege zu einem klimaneutralen Energiesystem. Die deutsche Energiewende im Kontext gesellschaftlicher Verhaltensweisen"., Freiburg.
* Fraunhofer ISE (2018): Stromgestehungskosten Erneuerbare Energien.

Steps applied:
* Read in data, strip and filter columns and do some renaming
* Store cost estimates in a dict indexed by energy sources
* Estimate LCOE based on the cost data from that dictionary as well as the full load hours and WACC assumptions from the 2018 LCOE study by Fraunhofer ISE
* Filter values later than the start year for the capacity expansion

In [ ]:
RES_cost_ISE = pd.read_csv(path_folder_inputs+path_folder_renewables+filename_RES_cost_ISE,
                           sep=";")

RES_cost_ISE.columns = RES_cost_ISE.columns.str.strip()
RES_cost_ISE[["Komponente", "Größe", "Einheit"]] = RES_cost_ISE[["Komponente", "Größe", "Einheit"]].apply(
    lambda x: x.str.strip())
RES_cost_ISE.fillna(method="ffill", inplace=True)

cond1 = RES_cost_ISE["Komponente"].isin(["Wind Offshore", "Wind Onshore", "Photovoltaik Freifläche Süd"])
cond2 = RES_cost_ISE["Größe"] != "Potenzial"

RES_cost_ISE = RES_cost_ISE.loc[cond1 & cond2].drop(columns="Einheit")

RES_cost_ISE.replace({"Wind Offshore": "windoffshore",
                      "Wind offshore": "windoffshore",
                      "Wind Onshore": "windonshore",
                      "Wind onshore": "windonshore",
                      "Photovoltaik Freifläche Süd": "solarPV",
                      "PV Freifläche (ab 2000 kWp)": "solarPV",
                      "Investition": "capex",
                      "M/O-Kosten": "opex_percentage",
                      "Lebensdauer": "unit_lifetime"},
                     inplace=True)

RES_cost_ISE.set_index(["Komponente", "Größe"], inplace=True)
RES_cost_ISE = RES_cost_ISE.T

RES_cost_ISE_dict = {}
for energy_source in RES_cost_ISE.columns.get_level_values(0).unique():
    RES_cost_ISE_dict[energy_source] = RES_cost_ISE[energy_source]

In [ ]:
LCOE_windonshore_ISE = eeg.estimate_lcoe_ise(windonshore_cost_estimates, RES_cost_ISE_dict, "windonshore")
LCOE_windoffshore_ISE = eeg.estimate_lcoe_ise(windoffshore_cost_estimates, RES_cost_ISE_dict, "windoffshore")
LCOE_solarPV_ISE = eeg.estimate_lcoe_ise(solarPV_cost_estimates, RES_cost_ISE_dict, "solarPV")

LCOE_windonshore_ISE = LCOE_windonshore_ISE.loc[
    LCOE_windonshore_ISE.index >= annual_cap_expansion.at['windonshore', 'start_year']]
LCOE_windoffshore_ISE = LCOE_windoffshore_ISE.loc[
    LCOE_windoffshore_ISE.index >= annual_cap_expansion.at['windoffshore', 'start_year']]
LCOE_solarPV_ISE = LCOE_solarPV_ISE.loc[
    LCOE_solarPV_ISE.index >= annual_cap_expansion.at['solarPV', 'start_year']]

Determine the split of the annual capacity to be installed for PV, i.e. the amount installed in tenders under the market premium scheme resp. the remainder installed under the FIT regime:
* For 2021 and 2022, the capacities from the tender procedure are given.
* For the remaining years, the volumes are taken from the EEG 2021.
    * In § 28a EEG 2021 the tender volumes for solarPV are given.
    * These will be separated into two segments. A second segments of larger rooftop and buildings PV will be eligible for market premium payments as well.
    * For solarPV tenders, it is assumed that plants will be operational two years from the tender year. Hence, there is a shift by twor years in the dict defining the tender capacities.

In [ ]:
capacity_split_solarPV = pd.DataFrame(index=pv_capacities_agg.index.union(LCOE_solarPV.index), 
                                      columns=['MP', 'FIT'])
capacity_split_solarPV.loc[:,'FIT'] = annual_cap_expansion.at['solarPV', 'capacity']

EEG_2021_tenders_dict = {2023: 1850+300,
                         2024: 1600+300,
                         2025: 1650+350,
                         2026: 1650+350,
                         2027: 1650+400,
                         2028: 1550+400,
                         2029: 1550+400,
                         2030: 1550+400}

capacity_split_solarPV.loc[2023:2030, 'MP'] = pd.Series(EEG_2021_tenders_dict).values
capacity_split_solarPV.loc[2023:2030, 'FIT'] = capacity_split_solarPV.loc[2023:2030, 'FIT'].sub(
    capacity_split_solarPV.loc[2023:2030, 'MP'])

for year in pv_capacities_agg.index.values:
    capacity_split_solarPV.at[year, 'FIT'] = pv_capacities_agg.at[year, 'capacity']
    capacity_split_solarPV.at[year, 'MP'] = solarPV_commissionings.at[year, 'capacity_awarded'] / 1000

Add the new-built plants to the data set:
* Create a capacity distribution determining the (capacity) distribution for the LCOE estimates.
* Add the plants groups to the DataFrme containing new-built RES units and their corresponding values applied

In [ ]:
onshore_cap_distribution_new = eeg.create_cap_distribution_from_beta(
    LCOE_windonshore_ISE, cols=['LCOE_low', 'LCOE_middle', 'LCOE_high'])
offshore_cap_distribution_new = eeg.create_cap_distribution_from_beta(
    LCOE_windoffshore_ISE, cols=['LCOE_low', 'LCOE_middle', 'LCOE_high'])
solarPV_cap_distribution_new = eeg.create_cap_distribution_from_beta(
    LCOE_solarPV_ISE, cols=['LCOE_low', 'LCOE_middle', 'LCOE_high'])

eeg_pps_de_windonshore_2030 = eeg.add_new_built_res(onshore_cap_distribution_new, eeg_pps_de_windonshore_2030, 
                                                    annual_cap_expansion, 'Wind_Onshore', indexed_by_years=False)
eeg_pps_de_windoffshore_2030 = eeg.add_new_built_res(offshore_cap_distribution_new, eeg_pps_de_windoffshore_2030, 
                                                    annual_cap_expansion, 'Wind_Offshore', indexed_by_years=False)
eeg_pps_de_solarPV_2030 = eeg.add_new_built_res(solarPV_cap_distribution_new, eeg_pps_de_solarPV_2030, 
                                                annual_cap_expansion, 'Solar', indexed_by_years=False)

Add solarPV units which are to be installed under the FIT scheme

In [ ]:
for year in capacity_split_solarPV.index.values:
    eeg_pps_de_solarPV_2030.loc['solar_PV_FIT_'+str(year),
                                ['capacity', 'energy_source', 
                                'support_scheme', 'commissioning_year']] = (
            [capacity_split_solarPV.at[year, 'FIT'] * 1000, 'Solar', 'FIT', year])

#### Combine the data sets and cluster
Steps applied:
* Append the artificial new built units to the 2030 RES data set.
* Perform the clustering (same as above).
* Write the results to csv resp. Excel.

In [ ]:
eeg_pps_de_2030 = pd.concat([eeg_pps_de_2030, eeg_pps_de_solarPV_2030,
                             eeg_pps_de_windonshore_2030, eeg_pps_de_windoffshore_2030])

ee_agg_2030 = eeg.create_ee_transformers(eeg_pps_de_2030, cluster_no=3)
ee_agg_2030['country'] = 'DE'
ee_agg_2030['to_el'] = 'DE_bus_el'
ee_agg_2030['efficiency_el'] = 1
buses_2030.extend(ee_agg_2030['from'].unique())

# seperate costs and nodes
costs_operation_RES_2030 = -ee_agg_2030['value_applied'].to_frame().rename(columns={'value_applied': 'costs'})

ee_agg_2030.to_csv(path_folder_outputs+filename_transformers_RES_2030)
ee_agg_2030.to_excel(writer, sheet_name='transformers_RES_2030')
costs_operation_RES_2030.to_csv(path_folder_outputs+filename_costs_operation_res_2030)

In [ ]:
del eeg_pps_de_2030

# Transformers and commodity sources for 2030 (addition)
## Add new-built power plants by assumption
**Background and approach**:

* The capacity estimation for 2030 showed that large amounts of capacities are about to be decommissioned whereas there is a limited number of plans for new-built power plants and the estimate of the TSOs from the NEP 2018 is rather conservative (i. e. not considering a coal phase-out and assuming fewer plants to go offline).
* In order not to provoke situations with a loss of load (LOL), the idea is to introduce backup power plants for levelling out the system balance.
* The scheme of the capacity balance (sheet) is used here. The latest TSO estimations covering 2018-2022 are used as a rough guideline. The scenario with a coal phase out is used. It shows that for 2021 with around 72 GW of installed capacity not including network reserve and security backup (of coal power plants), the remaining margin of 2 GW is small but sufficient. Hence, it is assumed that installed capacity has to add up to 72 GW in order to meet the peak demand.
* Accordignly, the difference between the overall capacity given here for 2030 (roughly 57 GW + 80% of phes and 28% of ROR capacities, 60% of biomass capacities and 1% of wind capacities) and the needed 72 GW is filled up with gas power plants.
* The lacking capacity is evenly distributed between gas turbines and CCGT power plants.

ÜNB (2020): Bericht zur Leistungsbilanz, https://www.netztransparenz.de/portals/1/Bericht_zur_Leistungsbilanz_2019.pdf, accessed 14.11.2020.

In [ ]:
cap_requirement_2030 = 72000

# Add all capacity credits to calculate secured capcity
secured_cap_2030 = conv_de_2030.capacity.sum() + conv_de_new.capacity.sum()
secured_cap_2030 += 0.28 * ror_de_agg.at['DE_source_ROR', 'capacity'] + 0.8 * phes_de_2030.capacity.sum()
secured_cap_2030 += 0.6 * sources_RES_2030.at['DE_source_biomassEEG', 'capacity']
secured_cap_2030 += 0.01 * res_de_projection_2030.loc[['windonshore', 'windoffshore'], 'capacity'].sum()

missing_cap_2030 = np.max([cap_requirement_2030 - secured_cap_2030, 0])
print(f'Capacity to be newly provided by additional gas power plants in 2030: {missing_cap_2030:,.2f} MW')

In [ ]:
transformers_new_built = pd.read_csv(path_folder_inputs+path_folder_pp_er+filename_new_built, 
                                     sep=";", decimal=",", index_col=0)

transformers_new_built['capacity'] = missing_cap_2030/2

if transformers_new_built.capacity.sum() == 0:
    pass
else:
    conv_2030 = conv_2030.append(transformers_new_built)

## Write transformer data for 2030

In [ ]:
conv_2030 = conv_2030.round(4)
conv_2030.to_csv(path_folder_outputs+filename_transformers_2030)
conv_2030.to_excel(writer, sheet_name='conv_2030')

## Commodity sources for 2030
Some combinations of fuel and bidding zone are only introduced in 2030. Other combinations are only active in the start year (2017 or 2019). In order not to introduce empty components which would be prone to modeling errors, the commodity sources and buses data sets are distincted.

In [ ]:
sources_commodity_2030 = conv_2030.reset_index()[['country', 'fuel', 'from']].drop_duplicates(subset='from')
sources_commodity_2030['label'] = sources_commodity_2030['country'] + '_source_' + sources_commodity_2030['fuel']
sources_commodity_2030 = sources_commodity_2030.merge(emission_factors, 
                                                      on='fuel').drop(columns='fuel').set_index('label')
sources_commodity_2030.rename(columns={'from': 'to'}, inplace=True)

sources_commodity_2030.to_csv(path_folder_outputs+filename_sources_commodity_2030)
sources_commodity_2030.to_excel(writer, sheet_name='sources_commodity_2030')

buses_2030.extend(sources_commodity_2030['to'].values)

costs_carbon_2030 = pd.DataFrame(index=sources_commodity_2030.index, columns=range(2015,2050), data=5.8)
costs_carbon_2030.to_csv(path_folder_outputs+filename_costs_carbon_2030)

# Sinks

In this section, **demand** data is put together.<br>
In [oemof.solph](https://github.com/oemof/oemof-solph) demand can be represented through (generic) so called "Sink" elements which have one input and no output. In addition to the demand sinks per bidding zone, excess sinks are created to model an overall generation surplus which cannot be consumed or stored within the area considered. In reality this would be exports to other European countries or result in curtailments. The excess sinks are needed for ensuring model feasibility.

## Electricity Demand

Currently the time series for electricity demand is created in the RES section, since the time series are loaded there.

Steps applied:
- Add bus information and label for demand
- Normalize demand time series by determining peak demand and dividing the demand timeseries through that value
- Write demand values (i.e. maximum demands) and demand time series to csv and to Excel files

In [ ]:
sinks_demand_el = pd.DataFrame(data={'country': countries_modeled})
sinks_demand_el['from'] = sinks_demand_el['country'] + '_bus_el'
sinks_demand_el['label'] = sinks_demand_el['country'] + '_sink_el_load'
sinks_demand_el.set_index('label', inplace = True)

In [ ]:
ts_eu_el_demand = ts_eu.filter(regex = 'sink_el')
peak_demand = ts_eu_el_demand.max()
sinks_demand_el.loc[peak_demand.index, 'maximum'] = peak_demand

sinks_demand_el_ts = ts_eu_el_demand/peak_demand

sinks_demand_el['maximum'] = sinks_demand_el['maximum'].astype(int)
sinks_demand_el_ts = sinks_demand_el_ts.round(4).astype(np.float32)

# interpolate NaN values (Czech and France)
sinks_demand_el_ts = sinks_demand_el_ts.interpolate(method='linear')

sinks_demand_el.to_csv(path_folder_outputs+filename_sinks_demand_el)
sinks_demand_el_ts.to_csv(path_folder_outputs+filename_sinks_demand_el_ts)
sinks_demand_el.to_excel(writer, sheet_name='sinks_demand_el')
sinks_demand_el_ts.tz_localize(None).to_excel(writer, sheet_name='sinks_demand_el_ts')

## Excess Sinks
Steps applied:
- Define and parameterize electricity excess sink per bidding zone
- Introduce high excess costs (penalty) costs in order not to produce to much excess
- Concatenate the electricity excess Sinks with the reservoir spillage Sinks
- Drop the German electricity excess Sink since it is explicitly modeled in _POMMES_
- Write the Sinks data to a csv and an Excel file

In [ ]:
sinks_excess = pd.DataFrame(data = {'country': countries_modeled})
sinks_excess['label'] = sinks_excess['country'] + '_sink_el_excess'
sinks_excess['from'] = sinks_excess['country'] + '_bus_el'
sinks_excess['excess_costs'] = 1000000
sinks_excess.set_index('label', inplace = True)

sinks_excess = pd.concat([sinks_excess, sinks_reservoir_spillage])
# The German electricity excess Sink is explicitly modeled in the model
sinks_excess.drop(index='DE_sink_el_excess', inplace=True)
sinks_excess.to_csv(path_folder_outputs+filename_sinks_excess)
sinks_excess.to_excel(writer, sheet_name='sinks_excess')

# Buses

Save all buses elements which were already created above to csv and to Excel
Links to sections were buses were created resp. adde:
- [electricity buses](#Introduce-electricity-buses) &rarr; instanciation of buses data set
- [hydro buses](#Prepare-data-for-usage-in-oemof.solph)
- [commodity buses](#Commodity-sources)

In [ ]:
# status quo
buses_df = pd.DataFrame(index=buses)
buses_df['country'] = [_[0] for _ in buses_df.index.str.split('_')]
buses_df.index.name = 'label'
buses_df.to_csv(path_folder_outputs+filename_buses)
buses_df.to_excel(writer, sheet_name='buses')

# 2030
buses_df_2030 = pd.DataFrame(index=buses_2030)
buses_df_2030['country'] = [_[0] for _ in buses_df_2030.index.str.split('_')]
buses_df_2030.index.name = 'label'
buses_df_2030.to_csv(path_folder_outputs+filename_buses_2030)
buses_df_2030.to_excel(writer, sheet_name='buses_2030')

# Save all DataFrames into one Excel file

In [ ]:
# Close the Pandas Excel writer and output the Excel file.
writer.save()

# Costs
Costs are read in from raw sheets and just spreaded here in order to fit the countries used.

Cost categories considered:
- OPEX for conventional power plants
- Ramping costs
- Fuel costs (commodity only)
- Storage OPEX

Steps applied for each of the cost categories:
- Read in the raw data
- Match index with data set (in most cases conventional power plants)
- Round values
- Save results to a csv file

In [ ]:
sources_commodity['fuel'] = [_[1] for _ in sources_commodity['to'].str.rsplit('_', 1)]
sources_commodity_2030['fuel'] = [_[1] for _ in sources_commodity_2030['to'].str.rsplit('_', 1)]
sources_commodity['technology'] = [_[1] for _ in sources_commodity['to'].str.rsplit('_', 1)]
sources_commodity_2030['technology'] = [_[1] for _ in sources_commodity_2030['to'].str.rsplit('_', 1)]

In [ ]:
opex = pd.read_excel(path_folder_inputs+path_folder_costs+filename_costs, 
                     sheet_name='operation_costs',
                     index_col=0)

costs_opex = opex.loc[sources_commodity['fuel'].values]
costs_opex.index = sources_commodity['to'].values
costs_opex.index.name = 'label'

costs_opex = costs_opex.round(2).astype(np.float32)

costs_opex_2030 = opex.loc[sources_commodity_2030['fuel'].values]
costs_opex_2030.index = sources_commodity_2030['to'].values
costs_opex_2030.index.name = 'label'

costs_opex_2030 = costs_opex_2030.round(2).astype(np.float32)

costs_opex.to_csv(path_folder_outputs+filename_costs_operation)
costs_opex_2030.to_csv(path_folder_outputs+filename_costs_operation_2030)

In [ ]:
ramping = pd.read_excel(path_folder_inputs+path_folder_costs+filename_costs, 
                        sheet_name='ramping_costs',
                        index_col=0)

costs_ramping = ramping.loc[sources_commodity['fuel'].values]
costs_ramping.index = sources_commodity['to'].values
costs_ramping.index.name = 'label'

# add fRES buses
costs_ramping = pd.concat([costs_ramping, 
                           pd.DataFrame(index=ee_agg['from'].unique(), columns=range(2015,2051), data=0)])

costs_ramping.to_csv(path_folder_outputs+filename_costs_ramping)

costs_ramping_2030 = ramping.loc[sources_commodity_2030['fuel'].values]
costs_ramping_2030.index = sources_commodity_2030['to'].values
costs_ramping_2030.index.name = 'label'

# add fRES buses
costs_ramping_2030 = pd.concat([costs_ramping_2030, 
                                pd.DataFrame(index=ee_agg_2030['from'].unique(), columns=range(2015,2051), data=0)])

costs_ramping_2030.to_csv(path_folder_outputs+filename_costs_ramping_2030)

In [ ]:
fuel = pd.read_excel(path_folder_inputs+path_folder_costs+filename_costs, 
                     sheet_name='middle_fuel_costs',
                     index_col=0)

costs_fuel = fuel.loc[sources_commodity['technology'].values]
costs_fuel.index = sources_commodity.index
costs_fuel.index.name = 'label'

costs_fuel = costs_fuel.round(2).astype(np.float32)
costs_fuel.to_csv(path_folder_outputs+filename_costs_fuel)

costs_fuel_2030 = fuel.loc[sources_commodity_2030['technology'].values]
costs_fuel_2030.index = sources_commodity_2030.index
costs_fuel_2030.index.name = 'label'

costs_fuel_2030 = costs_fuel_2030.round(2).astype(np.float32)
costs_fuel_2030.to_csv(path_folder_outputs+filename_costs_fuel_2030)

In [ ]:
opex_storages = pd.DataFrame(index=storages_el.index, columns=range(2015,2051), data=1)
opex_storages.to_csv(path_folder_outputs+filename_costs_operation_storage)

opex_storages_2030 = pd.DataFrame(index=storages_el_2030.index, columns=range(2015,2051), data=1)
opex_storages_2030.to_csv(path_folder_outputs+filename_costs_operation_storage_2030)